# Licenses

In [ ]:
#@title ##### GraphNets License { form-width: "50%" }
# Copyright 2018 The GraphNets Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================

In [ ]:
#@title ##### Modified Notice - MCMB Lab { form-width: "50%" }
# The GraphNets framework has been modified by:
# Multiscale Computational Mechanics and Biomechanics LAB (MCMB Lab)
# Department of Mechanical Engineering and Mechanics
# Drexel University, Philadelphia, Pennsylvania 
# Diroctor: Ahmad Raeisi Najafi (Ph.D.)
# All rights reserved.
# Developer: Nolan Black 10/2020
# Created in: 10/2020
# Last modified in: 2021
# ============================================================================

\
# Install Libraries On this Runtime


\


In [ ]:
#@title ### Install the Graph Nets library on this Colaboratory runtime  { form-width: "10%", run: "auto"}
#@markdown <br>1. Connect to a local or hosted Colaboratory runtime by clicking the **Connect** button at the top-right.<br>2. Choose "Yes" below to install the Graph Nets library on the runtime machine with the correct dependencies. Note, this works both with local and hosted Colaboratory runtimes.

install_graph_nets_library = "Yes"  #@param ["Yes", "No"]

if install_graph_nets_library.lower() == "yes":
  print("Installing Graph Nets library and dependencies:")
  print("Output message from command:\n")
  #!pip install graph_nets "dm-sonnet<2" "tensorflow_probability<0.9"
  !pip install graph_nets "tensorflow_gpu>=1.15,<2" "dm-sonnet<2" "tensorflow_probability<0.9"
else:
  print("Skipping installation of Graph Nets library")

Installing Graph Nets library and dependencies:
Output message from command:

     |████████████████████████████████| 81kB 3.3MB/s 
     |████████████████████████████████| 411.0MB 42kB/s 
     |████████████████████████████████| 665kB 21.9MB/s 
     |████████████████████████████████| 2.5MB 30.0MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 512kB 29.5MB/s 
     |████████████████████████████████| 2.9MB 33.3MB/s 
     |████████████████████████████████| 3.8MB 28.8MB/s 
  Created wheel for graph-nets: filename=graph_nets-1.1.0-cp37-none-any.whl size=91856 sha256=a845025aebe0c9c015436b1209db618d593b9a684929df0c7dbfd868d0704785
  Stored in directory: /root/.cache/pip/wheels/c9/85/dc/ad3c616f83f0774ff12d6d7599850f56d82976e8590e59e864
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=832f87d2683908d9333d8f5352316cffba5b0cc0900d0a3643277d9d699d66eb
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fceb

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from graph_nets import blocks
from graph_nets import utils_tf
from graph_nets import utils_np
from graph_nets import graphs
#from graph_nets.demos import models
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np
import sonnet as snt
import tensorflow as tf
import scipy
import scipy.stats as stats
import dill
# Optimization Params
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve as sparse_solve
# from scipy.linalg import solve 
from numpy.linalg import solve
import warnings
import copy

try:
  import seaborn as sns
except ImportError:
  pass
else:
  sns.reset_orig()

SEED = 1
np.random.seed(SEED)
tf.set_random_seed(SEED)
precision_base = 16 # to ease floating-point pains

print(f"Tensor Flow Version: {tf.__version__}")
gpu = tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None)
print("GPU is", "available" if gpu else "NOT AVAILABLE")

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Tensor Flow Version: 1.15.5
GPU is NOT AVAILABLE


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14522338480000962145, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11285393118811749789
 physical_device_desc: "device: XLA_CPU device"]

# **FEA Functions**
3 Node triangular element\
2nd order Gauss Quadrature Isoparametric mapping\
Homogeneous, Isotropic Plane Stress (2D)\

4 Node quadrilateral element\
2nd order Gauss Quadrature Isoparametric mapping\
Homogeneous, Isotropic Plane Stress (2D)\
aka CPS4, bilinear 2D Quad. Plane Stress







### Helper Structures

In [ ]:
class Mesh() :
  def __init__(self, dim, m1, m2, d1, d2, bound_x, bound_y, nodes_per_elem, n_elem, n_node, coords, elem_nodes) :
    self.dim = dim # dimension, scalar
    self.m1 = m1 # elements in x, scalar
    self.m2 = m2 # elements in y, scalar   
    self.d1 = d1 # element x width, float
    self.d2 = d2 # element y width, float
    self.bound_x = bound_x # [-x_lim, +x_lim]
    self.bound_y = bound_y # [-y_lim, +y_lim]
    self.nodes_per_elem = nodes_per_elem # scalar
    self.n_elem = n_elem # scalar
    self.n_node = n_node # scalar
    self.coords = coords # [ [x_i, y_i], ...] for n_node
    self.elem_nodes = elem_nodes # [ [n0, n1, n2, n3] .... ] for n_elem


class Material() :
  def __init__(self, E, v, elem_stiff, elem_density, Ce) :
    self.E = E # scalar
    self.v = v # scalar
    self.elem_stiff = elem_stiff # [ [E_i, v_i] ...] for n_elem
    self.elem_density = elem_density # [ p_i, ... ] for n_elem
    self.Ce = Ce # [ [Cmat], ...] for n_elem

class BC() :
  def __init__(self, n_pre_disp, n_dof, eq_num,  
               disp_dof, disp_node, disp_val,  
               force_dof, force_node, force_val, elem_load) :
    self.n_pre_disp = n_pre_disp # number of prescribed displacements
    self.n_dof = n_dof # number of free nodes
    self.eq_num = eq_num # list, defines free and prescribed nodes for matrix assembly
    self.disp_dof = disp_dof # [0, 1]
    self.disp_node = disp_node # node_id of prescribed disps
    self.disp_val = disp_val # presctibed disp value at each node_d
    self.force_dof = force_dof # force_dof at applied force nodes
    self.force_node = force_node # node_id of applied forces
    self.force_val = force_val # force val at each node_id
    self.elem_load = elem_load # [ [b_x, b_y] ... ] for n_elem, body force values

class FEA() :
  def __init__(self, UP, PF, PP, KPP, KPF, KFP, KFF, KUR) :
    self.UP = UP
    self.PF = PF
    self.PP = PP
    self.KPP = KPP
    self.KPF = KPF
    self.KFP = KFP
    self.KFF = KFF
    self.KUR = KUR


class Results() :
  def __init__(self, UF, R, UUR, PUR) :
    self.UF = UF
    self.R = R
    self.UUR = UUR
    self.PUR = PUR

### Mesh Noise

In [ ]:
def refine_mesh(refine_fract, coords, n_node, elem_nodes, n_elem) :
  """
  Refine a fraction of the existing elements to half their size. 
  TRIANGULAR ELEM ONLY.
  Returns Refined: coords, n_node, elem_node, n_elem
  """
  refined_coords = []
  already_refined = []
  
  num_refinements = int(refine_fract * n_elem / 2 ) 
  for i in range(num_refinements) :
    refine_elem = 2 * (np.random.randint(0, n_elem-1) // 2)
    while (True) :
      if (refine_elem not in already_refined ) :
        break
      else :
        refine_elem = 2 * (np.random.randint(0, n_elem) // 2)

    if i > 0 :
      for j, val in enumerate(already_refined) :
        if val > refine_elem :
            already_refined[j] += 2
        
    for j in range(4) :
      already_refined.append(refine_elem + j)


    elem_0 = elem_nodes[refine_elem]
    elem_1 = elem_nodes[refine_elem + 1]
    n_point = n_node
    
    n_x = (coords[elem_0[0]][0] + coords[elem_0[1]][0]) / 2
    n_y = (coords[elem_0[0]][1] + coords[elem_0[2]][1]) / 2
    coords.append([n_x, n_y])
    refined_coords.append([n_x, n_y])
    n_node +=1 

    elem_nodes.pop(refine_elem)
    elem_nodes.pop(refine_elem)
    if elem_0[1] == elem_0[0] + 1 :
      elem_nodes.insert(refine_elem, [n_point, elem_1[1], elem_1[2]])
      elem_nodes.insert(refine_elem, [n_point, elem_0[1], elem_1[1]])
      elem_nodes.insert(refine_elem, [elem_0[0], elem_0[1], n_point])
      elem_nodes.insert(refine_elem, [elem_0[0], n_point, elem_0[2]])
    else :
      elem_nodes.insert(refine_elem, [n_point, elem_0[1], elem_0[2]])
      elem_nodes.insert(refine_elem, [n_point, elem_1[1], elem_1[2]])
      elem_nodes.insert(refine_elem, [elem_1[0], elem_1[1], n_point])
      elem_nodes.insert(refine_elem, [elem_0[0], n_point, elem_0[2]])
    n_elem +=2

  return refined_coords, coords, n_node, elem_nodes, n_elem

def noise_mesh(elem_type, noise_type, noise_coord_scale,
              refined_coords, coords, m1, m2, d1, d2) :
  """
  Applies noise to the nodal positions.
  "posi" applies random uniform noise to the nodal posiions, 
    within a certain neighborhood defined by noise_coord_scale
  "row" does all "posi" noise and also squeezes a row/column pair
    to crudely change element size.
  
  Returns: Noisy coordinates in coords. 
  """

  
  if noise_type == "posi" :
    # Noise
    for i, point in enumerate(coords) :
      x, y = point[0], point[1]
      if [x, y] in refined_coords :
        if (x not in bound_x)  :
          x+= 0.5*noise_coord_scale*np.random.uniform(-d1, d1)
        if (y not in bound_y)  :
          y+= 0.5*noise_coord_scale*np.random.uniform(-d2, d2)
      else :
        if (x not in bound_x)  :
          x+= noise_coord_scale*np.random.uniform(-d1, d1)
        if (y not in bound_y)  :
          y+= noise_coord_scale*np.random.uniform(-d2, d2)
      coords[i] = [x, y]
  
  elif noise_type == "row" :
    # Shift mesh
    target_row = np.random.randint(2,m2-2)
    target_col = np.random.randint(2,m1-2)
    
    shifted_x = []
    shifted_y = []
    for j in range(-1,2,2) :
      fact = (-j//2)
      if j < 0 :
          fact +=1
      for i in range(m2 + 1) :
        node = i*(m1 + 1) + target_col -fact
        coords[node][0] += fact*d1*.5
        shifted_x.append(coords[node][0])
      start = (target_row-fact)*(m1 + 1)
      end = start + m1  +1
      for node in range(start, end) :
        coords[node][1] += fact*d2*.5
        shifted_y.append(coords[node][1])
    
    # Noise
    for i, point in enumerate(coords) :
      x, y = point[0], point[1]
      if (x not in bound_x) and (x not in shifted_x) :
        x+= noise_coord_scale*np.random.uniform(-d1, d1)
      elif (x not in bound_x) and (x in shifted_x):
        x+= 0.1*noise_coord_scale*np.random.uniform(-d1, d1)
      if (y not in bound_y) and (y not in shifted_y) :
        y+= noise_coord_scale*np.random.uniform(-d2, d2)
      elif (y not in bound_y) and (y in shifted_y):
        y+= 0.1*noise_coord_scale*np.random.uniform(-d2, d2)
      coords[i] = [x, y]

  return coords

def flat_list(list_of_lists) :
  return [item for sublist in list_of_lists for item in sublist]

def noise_mesh_MS(noise_type, noise_coord_scale, mesh_0, mesh_1) :
  """
  Applies noise to the nodal positions.
  "posi" applies random uniform noise to the nodal posiions, 
    within a certain neighborhood defined by noise_coord_scale
  "row" does all "posi" noise and also squeezes a row/column pair
    to crudely change element size.

  Does not change coords of the coarse mesh (mesh_0).
  ONLY affects fine mesh (mesh_1)
  
  Returns: Noisy coordinates in coords. 
  """
  coord_precision = 7
  coords = copy.deepcopy(mesh_1.coords)
  off_limit_x = [round(point[0], coord_precision) for point in mesh_0.coords] + list(mesh_0.bound_x)
  off_limit_y = [round(point[1], coord_precision) for point in mesh_0.coords] + list(mesh_0.bound_y)

  if noise_type == "posi" :
    # Noise
    for i, point in enumerate(mesh_1.coords) :
      x, y = round(point[0], coord_precision), round(point[1], coord_precision)
      if (x not in off_limit_x) and (y not in off_limit_y) :
        x+= 1.0*noise_coord_scale*np.random.uniform(-mesh_1.d1, mesh_1.d1)
        y+= 1.0*noise_coord_scale*np.random.uniform(-mesh_1.d2, mesh_1.d2)
      mesh_1.coords[i] = [x, y]
  
  if noise_type == "row" :
    # Shift mesh
    m1_fact = mesh_1.m1 / mesh_0.m1
    m2_fact = mesh_1.m2 / mesh_0.m2
    # off_limits_m1 = []
    # for i in range(0, mesh_1.m1, m1_fact) :
    #   off_limits_m1

    target_row = np.random.randint(2,mesh_1.m2-2)
    target_col = np.random.randint(2,mesh_1.m1-2)
    off_limit_range_m1 = [target_col + 1, target_col,  target_col -1]
    off_limit_range_m2 = [target_row + 1, target_row, target_row -1]
    while any([(val) % m2_fact == 0 for val in off_limit_range_m2]):
      target_row = np.random.randint(2,mesh_1.m2-2)
      off_limit_range_m2 = [target_row + 1, target_row, target_row -1]
    while any([(val) % m1_fact ==0 for val in off_limit_range_m1]) :
      target_col = np.random.randint(2,mesh_1.m1-2)
      off_limit_range_m1 = [target_col + 1, target_col,  target_col -1]
    
    shifted_x = []
    shifted_y = []
    for j in range(-1,2,2) :
      fact = (-j//2)
      if j < 0 :
          fact +=1
      for i in range(mesh_1.m2 + 1) :
        node = i*(mesh_1.m1 + 1) + target_col -fact
        mesh_1.coords[node][0] += fact*mesh_1.d1*.5
        shifted_x.append(mesh_1.coords[node][0])
      start = (target_row-fact)*(mesh_1.m1 + 1)
      end = start + mesh_1.m1  +1
      for node in range(start, end) :
        mesh_1.coords[node][1] += fact*mesh_1.d2*.5
        shifted_y.append(mesh_1.coords[node][1])

    # Coord Noise
    for i, point in enumerate(mesh_1.coords) :
      x, y = round(point[0], coord_precision), round(point[1], coord_precision)
      if (x not in off_limit_x) :
        if x in shifted_x :
          x+= 0.0*noise_coord_scale*np.random.uniform(-mesh_1.d1, mesh_1.d1)
        else :
          x+= 1.0*noise_coord_scale*np.random.uniform(-mesh_1.d1, mesh_1.d1)
      if (y not in off_limit_y) :
        if y in shifted_y :
          y+= 0.0*noise_coord_scale*np.random.uniform(-mesh_1.d2, mesh_1.d2)
        else :
          y+= 1.0*noise_coord_scale*np.random.uniform(-mesh_1.d2, mesh_1.d2)
      mesh_1.coords[i] = [x, y]

  return mesh_1.coords

### FEA and Gauss Integration

In [ ]:
def shape_funct(elem_type, r) :
  if elem_type == "tri" :
    N = [1 - r[0] - r[1], r[0], r[1]]
    DN = [[-1, -1], [1, 0], [0, 1]]
    return np.array(N), np.array(DN) 
  else :
    N = [(1 - r[0])*(1 - r[1]),
            (1 + r[0])*(1 - r[1]),
            (1 + r[0])*(1 + r[1]),
            (1 - r[0])*(1 + r[1])]
    DN = [[-(1-r[1]), -(1-r[0])],
              [(1-r[1]), -(1+r[0])], 
              [(1+r[1]), (1+r[0])], 
              [-(1+r[1]), (1-r[0])]]

    return 0.25 * np.array(N),  0.25 * np.array(DN)         

def GaussQuad(elem_type) :
  """
  2nd order gauss quadrature for FEA calculations.
  4 Node quadraleteral uses 4 interior integration points.
  3 Node triangular uses 3 vertices as integration points.
  Returns: points and weights for summation.
  """
  if elem_type == "tri" :
    r = [[1/6, 1/6],
        [2/3, 1/6],
        [1/6, 2/3]]
    w = 1/3 + np.zeros((1,3))
  else :
    r = [[-1/np.math.sqrt(3), -1/np.math.sqrt(3)],
        [1/np.math.sqrt(3), -1/np.math.sqrt(3)],
        [1/np.math.sqrt(3),  1/np.math.sqrt(3)],
        [-1/np.math.sqrt(3),  1/np.math.sqrt(3)]]
    w = 1+ np.zeros((1,4))

  return np.array(r), w

def element(elem_type, elem_iter, mesh, material, bc) :
  """ 
  Applies shape functions and Gauss Quadrature
  Returns: Element stiffness matrix, element force matrix
  """
  r,w = GaussQuad(elem_type)
  
  if elem_type == "tri" :
    K_el = np.zeros((6,6))
    P_el = np.zeros((6,))
    x_el = [mesh.coords[mesh.elem_nodes[elem_iter][0]],
            mesh.coords[mesh.elem_nodes[elem_iter][1]],
            mesh.coords[mesh.elem_nodes[elem_iter][2]]]
    x_el = np.transpose(np.array(x_el))
    b = np.transpose(np.array(bc.elem_load[0:2,elem_iter]))

    for i in range(r.size//2) :
      N, DN = shape_funct(elem_type, r[i])
      J = np.matmul(x_el, DN)
      detJ = J[0,0]*J[1,1] - J[0,1]*J[1,0]
      invJ = np.matrix([[J[1,1], -J[0,1]], [-J[1,0], J[0,0]]])/detJ
      B = np.matmul(DN, invJ)
      NhatToI = np.kron(np.transpose(N), np.eye(2))
      BhatToI = np.zeros((3,6))
      BhatToI[0,0] = B[0][0,0]
      BhatToI[1,1] = B[0][0,1]
      BhatToI[0,2] = B[1][0,0]
      BhatToI[1,3] = B[1][0,1]
      BhatToI[0,4] = B[2][0,0]
      BhatToI[1,5] = B[2][0,1]
      
      BhatToI[2,0] = BhatToI[1,1]
      BhatToI[2,1] = BhatToI[0,0]
      BhatToI[2,2] = BhatToI[1,3]
      BhatToI[2,3] = BhatToI[0,2]
      BhatToI[2,4] = BhatToI[1,5]
      BhatToI[2,5] = BhatToI[0,4]

      K_el += 0.5*np.matmul(np.transpose(BhatToI),
                        np.matmul(material.Ce[elem_iter], BhatToI)) * detJ * w[0,i]
      P_el += np.matmul(np.transpose(NhatToI), b) * detJ * w[0,i]


  else :
    K_el = np.zeros((8,8))
    P_el = np.zeros((8,))
    x_el = [mesh.coords[mesh.elem_nodes[elem_iter][0]],
            mesh.coords[mesh.elem_nodes[elem_iter][1]],
            mesh.coords[mesh.elem_nodes[elem_iter][2]],
            mesh.coords[mesh.elem_nodes[elem_iter][3]]]
    x_el = np.transpose(np.array(x_el))
    b = np.transpose(np.array(bc.elem_load[0:2,elem_iter]))

    for i in range(r.size//2) :
      N, DN = shape_funct(elem_type, r[i])
      J = np.matmul(x_el, DN)
      detJ = J[0,0]*J[1,1] - J[0,1]*J[1,0]
      B = np.matmul(DN, np.matrix([[J[1,1], -J[0,1]], [-J[1,0], J[0,0]]])/detJ)
      NhatToI = np.kron(np.transpose(N), np.eye(2))
      BhatToI = np.kron(np.transpose(B), np.eye(2))
      K_el += np.matmul(np.transpose(BhatToI),
                        np.matmul(material.Ce[elem_iter], BhatToI)) * detJ * w[0,i]
      P_el += np.matmul(np.transpose(NhatToI), b) * detJ * w[0,i]
    
  return K_el, P_el


In [1]:
def assign_supports(support_option, mesh) :
  
  coord_precision = 5

  if support_option == "left_cantilever" :
    n_pre_disp = 0
    disp_node = []
    for i, point in enumerate(mesh.coords) :
      if round(point[0],coord_precision) == mesh.bound_x[0] :
        for j in range(mesh.dim) :
          disp_node.append(i)
          n_pre_disp += 1
  
  if support_option == "right_cantilever" :
    n_pre_disp = 0
    disp_node = []
    for i, point in enumerate(mesh.coords) :
      if round(point[0],coord_precision) == mesh.bound_x[1] :
        for j in range(mesh.dim) :
          disp_node.append(i)
          n_pre_disp += 1
  
  if support_option == "dual_cantilever" :
    n_pre_disp = 0
    disp_node = []
    for i, point in enumerate(mesh.coords) :
      if round(point[0],coord_precision) == mesh.bound_x[0] or round(point[0], coord_precision) == mesh.bound_x[1] :
        for j in range(mesh.dim) :
          disp_node.append(i)
          n_pre_disp += 1

  if support_option == "MBB" :
    n_pre_disp = 0
    disp_node = []
    for i, point in enumerate(mesh.coords) :
      if round(point[1], coord_precision) == mesh.bound_y[0] and ( round(point[0], coord_precision)  == mesh.bound_x[0] or round(point[0], coord_precision)  == mesh.bound_x[1] ):
        for j in range(mesh.dim) :
          disp_node.append(i)
          n_pre_disp += 1
  
  if support_option == "L_bracket" :
    n_pre_disp = 0
    disp_node = []
    for i, point in enumerate(mesh.coords) :
      if round(point[1], coord_precision) == (mesh.d2*(mesh.m1 + mesh.m2) + mesh.bound_y[0]) :
        for j in range(mesh.dim) :
          disp_node.append(i)
          n_pre_disp += 1

  disp_dof = []
  disp_val = []

  if support_option == "left_slider" or support_option == "right_slider" or support_option == "shear_slider" or support_option == "twist_slider":
    disp_node = []
    n_pre_disp = 0
    m1, m2 = mesh.m1, mesh.m2
    if support_option == "left_slider" :
      prescribed_disp_x = -0.2
      prescribed_disp_y = 0.0
    elif support_option == "right_slider" :
      prescribed_disp_x = 0.2
      prescribed_disp_y = 0.0
    elif support_option == "shear_slider" :
      prescribed_disp_x = 0.0
      prescribed_disp_y = 0.2
    else :
      prescribed_disp_x = 0.2
      prescribed_disp_y = 0.2
    # at left border
    start = 0
    increment = m1 + 1
    end = start + increment*(m2+1)
    for i in range(start, end, increment) :
      # prescribe x
      disp_node.append(i)
      disp_dof.append(0)
      disp_val.append(-prescribed_disp_x/2)
      n_pre_disp += 1
      # constrain y
      disp_node.append(i)
      disp_dof.append(1)
      disp_val.append(-prescribed_disp_y/2)
      n_pre_disp += 1

    # at right  border
    start = m1
    increment = m1 + 1
    end = start + increment*(m2+1)
    for i in range(start, end, increment) :
      # prescribe x
      disp_node.append(i)
      disp_dof.append(0)
      disp_val.append(prescribed_disp_x/2)
      n_pre_disp += 1
      # constrain y
      disp_node.append(i)
      disp_dof.append(1)
      disp_val.append(prescribed_disp_y/2)
      n_pre_disp += 1
  else :
    for i in range(0, n_pre_disp, mesh.dim) :
      disp_dof.append(0)
      disp_dof.append(1)
      disp_val.append(0)
      disp_val.append(0)
  return n_pre_disp, disp_node, disp_dof, disp_val

def assign_force(support_option, force_option, mesh, applied_force_dofs, applied_force_vals) :

  force_dof = []
  force_node = []
  force_val = []

  if support_option == "left_cantilever" or support_option == "L_bracket":
    node_shift = mesh.m1
  elif support_option == "MBB" :
    node_shift = np.random.choice([0, mesh.m1])
  else :
    node_shift = 0

  if force_option == "side_bottom" :
    n_force = 6
    for i in range(int(n_force/2)) :
      fnode = int(node_shift + (mesh.m1 + 1)*(i+1))
      for j in range(mesh.dim) :
        force_dof.append(applied_force_dofs[j])
        force_val.append(applied_force_vals[j])
        force_node.append(fnode)
    for i in range(len(force_val)) :
      force_val[i] /= (n_force/2) 

  if force_option == "side_top" :
    n_force = 6
    for i in range(int(n_force/2)) :
      fnode = int(node_shift+ (mesh.m2-3 + i)*(mesh.m1 + 1))
      for j in range(mesh.dim) :
        force_dof.append(applied_force_dofs[j])
        force_val.append(applied_force_vals[j])
        force_node.append(fnode)
    for i in range(len(force_val)) :
      force_val[i] /= (n_force/2) 

  if force_option == "side_center" :
    n_force = 6
    for i in range(3) :
      fnode = int(node_shift + (mesh.m1 + 1)*(mesh.m2/2 - 1 + i))
      for j in range(mesh.dim) :
        force_dof.append(applied_force_dofs[j])
        force_val.append(applied_force_vals[j])
        force_node.append(fnode)
    for i in range(len(force_val)) :
      force_val[i] /= 3    

  if force_option == "top_center" :
    n_force = 6
    for i in range(3) :
      fnode = int( (mesh.m1+1)*(mesh.m2+1) - mesh.m1/2 - 2 + i)
      for j in range(mesh.dim) :
        force_dof.append(applied_force_dofs[j])
        force_val.append(applied_force_vals[j])
        force_node.append(fnode)
    for i in range(len(force_val)) :
      force_val[i] /= 3   

  if force_option == "top_distributed" :
    n_force = 2*(mesh.m1 - 1)
    for i in range(int(n_force/2)) :
      fnode = int( (mesh.m1+1)*(mesh.m2+1) - mesh.m1 + i)
      for j in range(mesh.dim) :
        force_dof.append(applied_force_dofs[j])
        force_val.append(applied_force_vals[j])
        force_node.append(fnode)
    for i in range(len(force_val)) :
      force_val[i] /= (n_force/2)   

  return force_dof, force_node, force_val

def assign_BC_option(mesh, applied_force_vals, applied_force_dofs, body_force,
                     support_option, force_option) :

  # Displacement BC - !! count x and y separateley !!
  n_pre_disp, disp_node, disp_dof, disp_val = assign_supports(support_option, mesh)

 #################################################################

  # Body Forces
  elem_load = np.zeros((2, mesh.n_elem))
  for i in range(mesh.n_elem) :
    elem_load[0, i] = body_force[0]
    elem_load[1, i] = body_force[1]

  force_dof, force_node, force_val = assign_force(support_option, force_option, mesh, applied_force_dofs, applied_force_vals) 


  #################################################################
  # Processing
  # assign negative equation numnbes for nodes with prescribed displacements, else positive
  eq_num = np.zeros((2, mesh.n_node))
  for i in range(n_pre_disp) :
      node = disp_node[i]
      dof = disp_dof[i]
      eq_num[dof,node] = -i -1
  row = 0
  for i in range(mesh.n_node) :
      for j in range(2) :
          if eq_num[j,i] == 0 :
              row += 1
              eq_num[j,i] = row;
  n_dof = row

  return BC(n_pre_disp, n_dof, eq_num, disp_dof, disp_node, disp_val, force_dof, force_node, force_val, elem_load)

def build_mesh_L_bracket(elem_type, m1, m2, bound_x, bound_y, 
               noise_type, noise_coord_scale, refine_fract) :
  # Building Mesh
  precision_base = 16
  dim = 2 # i.e. 2d
  nodes_per_elem = 4
  d1 = (bound_x[1] - bound_x[0])/m1
  d2 = (bound_y[1] - bound_y[0])/m2
  nodes_per_elem = 4
  n_elem = 2*m1 * m2
  n_node = 2*(m1 + 1) * (m2 + 1) - (m2 + 1)

  # count coords row-wise, bottom to top, then apply noise
  coords = []
  for j in range(m2 + 1) :
    for i in range(m1 +1) :
      x = round(bound_x[0] + d1*i, precision_base)
      y = round(bound_y[0] + d2*j, precision_base)
      coords.append([x, y])
  # Add addition to L bracket
  for j in range(1, m1 + 1) :
    for i in range(m2 +1) :
      x = round(bound_x[0] + d1*i, precision_base)
      y = round(bound_y[1] + d2*j, precision_base)
      coords.append([x, y])

  # Build elements ccw from bottom left
  elem_nodes = []
  for i in range(0, (m2-1) * (m1 + 1) + 1, m1 + 1) :
    for j in range(i, i + m1) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1)
      elem_node.append(j + m1 + 2)
      elem_node.append(j + m1 + 1)
      elem_nodes.append(elem_node)
  # Add connection to L bracket
  for i in range(m2*(m1 + 1), m2*(m1 + 1) + m2 + 1, m2+1) :
    for j in range(i, i + m2) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1)
      elem_node.append(j + m1 + 2)
      elem_node.append(j + m1 + 1)
      elem_nodes.append(elem_node)
  # Add new material to L bracket
  for i in range((m2+1)*(m1 + 1), (m2+1)*(m1 + 1) + m2*(m1 + 1)-1, m2 + 1) :
    for j in range(i, i + m2) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1)
      elem_node.append(j + m2 + 2)
      elem_node.append(j + m2 + 1)
      elem_nodes.append(elem_node)

  # Refine Mesh
  refined_coords = []

  # Apply noise to coordinates
  coords = noise_mesh(elem_type, noise_type, noise_coord_scale,
                      refined_coords, coords, m1, m2, d1, d2)

  return Mesh(dim, m1, m2, d1, d2, bound_x, bound_y, nodes_per_elem, n_elem, n_node, coords, elem_nodes)

def build_mesh(elem_type, m1, m2, bound_x, bound_y, 
               noise_type, noise_coord_scale, refine_fract) :
  # Building Mesh
  precision_base = 16
  dim = 2 # i.e. 2d
  nodes_per_elem = 4
  d1 = (bound_x[1] - bound_x[0])/m1
  d2 = (bound_y[1] - bound_y[0])/m2
  if elem_type == "tri" :
    nodes_per_elem = 3
    n_elem = 2 * m1 * m2
  else :
    nodes_per_elem = 4
    n_elem = m1 * m2
  n_node = (m1 + 1) * (m2 + 1)

  # count coords row-wise, bottom to top, then apply noise
  coords = []
  for j in range(m2 + 1) :
    for i in range(m1 +1) :
      x = round(bound_x[0] + d1*i, precision_base)
      y = round(bound_y[0] + d2*j, precision_base)
      coords.append([x, y])

  # Build elements ccw from bottom left
  # Build elements ccw from bottom left
  elem_nodes = []
  diag = 0
  for i in range(0, (m2-1) * (m1 + 1) + 1, m1 + 1) :
    for j in range(i, i + m1) :
      if elem_type == "tri" :
        if (diag % 2) == 0 :
          elem_node = []
          elem_node.append(j)
          elem_node.append(j + 1)
          elem_node.append(j + m1 + 1)
          elem_nodes.append(elem_node)
          elem_node = []
          elem_node.append(j + 1)
          elem_node.append(j + m1 + 2)
          elem_node.append(j + m1 + 1)
          elem_nodes.append(elem_node)
        else :
          elem_node = []
          elem_node.append(j)
          elem_node.append(j + m1 + 2)
          elem_node.append(j + m1 + 1)
          elem_nodes.append(elem_node)
          elem_node = []
          elem_node.append(j)
          elem_node.append(j + 1)
          elem_node.append(j + m1 + 2)
          elem_nodes.append(elem_node)
        diag += 1
      else :
        elem_node = []
        elem_node.append(j)
        elem_node.append(j + 1)
        elem_node.append(j + m1 + 2)
        elem_node.append(j + m1 + 1)
        elem_nodes.append(elem_node)
    diag += (m1%2) + 1

  # Refine Mesh
  if elem_type == "tri" :
    refined_coords, coords, n_node, elem_nodes, n_elem = refine_mesh(
        refine_fract, coords, n_node, elem_nodes, n_elem)
  else :
    refined_coords = []

  # Apply noise to coordinates
  coords = noise_mesh(elem_type, noise_type, noise_coord_scale,
                      refined_coords, coords, m1, m2, d1, d2)

  return Mesh(dim, m1, m2, d1, d2, bound_x, bound_y, nodes_per_elem, n_elem, n_node, coords, elem_nodes)

def build_mesh_hole(elem_type, m1, m2, bound_x, bound_y, 
               noise_type, noise_coord_scale, refine_fract,
               center = [1.0, 0.0], radius = 0.0) :
  # Building Mesh
  precision_base = 8
  dim = 2 # i.e. 2d
  nodes_per_elem = 4
  d1 = (bound_x[1] - bound_x[0])/m1
  d2 = (bound_y[1] - bound_y[0])/m2
  nodes_per_elem = 4
  n_elem = m1 * m2
  n_node = (m1 + 1) * (m2 + 1)
  hole_density = [1]*n_elem


  # count coords row-wise, bottom to top, then apply noise
  # hole_bound_x = [center[0]-radius, center[0]+radius]
  # hole_bound_y = [center[1]-radius, center[1]+radius]
  # border_bound_x = [center[0]-radius-d1, center[0]+radius+d1]
  # border_bound_y = [center[1]-radius-d2, center[1]+radius+d2]
  border_bound_x = [center[0]-radius, center[0]+radius]
  border_bound_y = [center[1]-radius, center[1]+radius]
  hole_bound_x = [center[0]-radius+d1, center[0]+radius-d1]
  hole_bound_y = [center[1]-radius+d2, center[1]+radius-d2]
  node_i = 0
  coords = []
  skip_nodes = []
  border_nodes = []
  for j in range(m2 + 1) :
    for i in range(m1 +1) :
      x = round(bound_x[0] + d1*i, precision_base)
      y = round(bound_y[0] + d2*j, precision_base)
      if ((x >= hole_bound_x[0]) and (x <= hole_bound_x[1]) and 
          (y >= hole_bound_y[0]) and (y <= hole_bound_y[1])) :
        skip_nodes.append(node_i)
        coords.append([x, y])
        # n_node -= 1
      elif ((x >= border_bound_x[0]) and (x <= border_bound_x[1]) and 
        (y >= border_bound_y[0]) and (y <= border_bound_y[1])) :
        border_nodes.append(node_i)
        try :
          theta = np.arctan(np.abs( (y-center[1])/(x-center[0])))
        except :
          theta = np.pi/2
        x_delta = radius*np.cos(theta)
        y_delta = radius*np.sin(theta)
        if x <= center[0] : 
          x = center[0] - x_delta
        else :
          x = center[0] + x_delta
        if y <= center[1] :
          y = center[1] - y_delta
        else :
          y = center[1] + y_delta
        coords.append([x, y])
      else :
        coords.append([x, y])
      node_i += 1


  # Build elements ccw from bottom left
  # Build elements ccw from bottom left
  elem_nodes = []
  elem_i = 0
  for i in range(0, (m2-1) * (m1 + 1) + 1, m1 + 1) :
    for j in range(i, i + m1) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1)
      elem_node.append(j + m1 + 2)
      elem_node.append(j + m1 + 1)
      if all(i in border_nodes+skip_nodes for i in elem_node) :
        hole_density[elem_i] = 1e-6
      elem_nodes.append(elem_node)
      elem_i += 1

  # Refine Mesh
  refined_coords = []

  # Apply noise to coordinates
  coords = noise_mesh(elem_type, noise_type, noise_coord_scale,
                      refined_coords, coords, m1, m2, d1, d2)
  
  mesh = Mesh(dim, m1, m2, d1, d2, bound_x, bound_y, nodes_per_elem, n_elem, n_node, coords, elem_nodes)

  return mesh, hole_density


def dual_mesh_pre_process_L_bracket(mesh_0, mesh_1) :
  M1 = mesh_0.m1
  M2 = mesh_0.m2
  m1 = mesh_1.m1
  m2 = mesh_1.m2
  m1_fact = int(m1/M1)
  m2_fact = int(m2/M2)

  father_elems = []
  for i in range(0, (m2-1) * (m1 + 1) + 1, m2_fact*(m1 + 1)) :
    for j in range(i, (i + m1), m1_fact) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1*m1_fact)
      elem_node.append(j + m1*m2_fact + m1_fact + m2_fact)
      elem_node.append(j + m1*m2_fact + m2_fact)
      father_elems.append(elem_node)
  # Add connection for L bracket
  for i in range(m2*(m1 + 1), m2*(m1 + 1) + m2 + 1, m2_fact*(m2+1)) :
    for j in range(i, i + m2, m1_fact) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + m1_fact)
      elem_node.append(j + (m1+1) + (m2_fact-1)*(m2+1) + m1_fact)
      elem_node.append(j + (m1+1) + (m2_fact-1)*(m2+1))
      father_elems.append(elem_node)
  # Add new material to L bracket
  for i in range((m2+1)*(m1 + 1) + (m2_fact-1)*(m2+1), (m2+1)*(m1 + 1) + m2*(m1 + 1)-1, m2_fact*(m2+1)) :
    for j in range(i, i + m2, m1_fact) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1*m1_fact)
      elem_node.append(j + m2*m2_fact + m1_fact + m2_fact)
      elem_node.append(j + m2*m2_fact + m2_fact)
      father_elems.append(elem_node)

  graph_nodes = []
  for elem in father_elems[0:M1*M2] :
    graph_node = []
    for i in range(elem[0], elem[2] + 1, m1+1) :
      for j in range(i, i + 1 + m1_fact) :
        graph_node.append(j)
    graph_nodes.append(graph_node)

  # add connection for L bracket
  for elem in father_elems[(M1*M2):((M1*M2) + M2)] :
    graph_node = []
    for i in range(elem[0], elem[0] + 2*(m1+1)-1, m1+1) :
      for j in range(i, i + 1 + m2_fact) :
        graph_node.append(j)
    for i in range(elem[0] + (m1+1) + (m2+1), elem[2] +1, m2+1) :
      for j in range(i, i + 1 + m2_fact) :
        graph_node.append(j)
    graph_nodes.append(graph_node)

  # add material for L bracket
  for elem in father_elems[(M1*M2) + M2::] :
    graph_node = []
    for i in range(elem[0], elem[2] + 1, m2+1) :
      for j in range(i, i + 1 + m2_fact) :
        graph_node.append(j)
    graph_nodes.append(graph_node)

  return father_elems, graph_nodes

def assign_material(elem_type, mesh, E, v, elem_density) :
  # Material Assignment
  # Homogeneous, Isotropic Plane Stress
  elem_stiff = np.zeros((2, mesh.n_elem))
  for i in range(mesh.n_elem) :
    elem_stiff[0, i] = E
    elem_stiff[1, i] = v
    
  Ce = []
  for i in range(mesh.n_elem) :
    E_i = elem_stiff[0, i]
    v_i = elem_stiff[1, i]
    mu = 0.5*E_i/(1+v_i)
    lam = E_i*v_i/((1+v_i)*(1-2*v_i))
    if elem_type == "tri" :
      C = (E_i/(1-v_i**2)) * np.matrix([[1, v_i, 0],
                                [v_i, 1, 0],
                                [0, 0, (1-v_i)/2]])
    else : 
      C = np.matrix([[lam+2*mu, 0, 0, lam],
                    [0, mu, mu, 0],
                    [0, mu, mu, 0],
                    [lam, 0, 0, lam+2*mu]])
    Ce.append(C)
  
  # Assign element density
  for i in range(mesh.n_elem) :
    Ce[i] = Ce[i] * elem_density[i]

  return Material(E, v, elem_stiff, elem_density, Ce)


def assign_BC(mesh, applied_force_vals, applied_force_dofs, body_force) :
  
  """ Loading and BC for a cantilever """
  
  # Displacement BC - !! count x and y separateley !!
  n_pre_disp = 0
  disp_node = []
  for i, point in enumerate(mesh.coords) :
    if point[0] == mesh.bound_x[0] :
      for j in range(mesh.dim) :
        disp_node.append(i)
        n_pre_disp += 1

  disp_dof = []
  disp_val = []
  for i in range(0, n_pre_disp, mesh.dim) :
    disp_dof.append(0)
    disp_dof.append(1)
    disp_val.append(0)
    disp_val.append(0)

#################################################################

  # Body Forces
  elem_load = np.zeros((2, mesh.n_elem))
  for i in range(mesh.n_elem) :
    elem_load[0, i] = body_force[0]
    elem_load[1, i] = body_force[1]

  # Nodal Forces - !! count F_x and F_y separateley !!
  force_dof = []
  force_node = []
  force_val = []
  n_force = 6
  for i in range(3) :
    fnode = int(mesh.m1 + (mesh.m1 + 1)*(mesh.m2/2 - 1 + i))
    for j in range(mesh.dim) :
      force_dof.append(applied_force_dofs[j])
      force_val.append(applied_force_vals[j])
      force_node.append(fnode)
  for i in range(len(force_val)) :
    force_val[i] /= 3    

  # Bottom corner force:
  # n_force = 2
  # force_dof = []
  # force_node = []
  # force_val = []
  # for i in range(1) :
  #   fnode = int(m1 + (m1 + 1)*(i))
  #   for j in range(dim) :
  #     force_dof.append(force_dofs[j])
  #     force_val.append(force_vals[j])
  #     force_node.append(fnode)
  # for i in range(len(force_val)) :
  #   force_val[i] /= 1    


  #################################################################
  # Processing
  # assign negative equation numnbes for nodes with prescribed displacements, else positive
  eq_num = np.zeros((2, mesh.n_node))
  for i in range(n_pre_disp) :
      node = disp_node[i]
      dof = disp_dof[i]
      eq_num[dof,node] = -i -1
  row = 0
  for i in range(mesh.n_node) :
      for j in range(2) :
          if eq_num[j,i] == 0 :
              row += 1
              eq_num[j,i] = row;
  n_dof = row

  return BC(n_pre_disp, n_dof, eq_num, disp_dof, disp_node, disp_val, force_dof, force_node, force_val, elem_load)

def assemble_matrices(elem_type, mesh, material, bc, efficient_opt) :
  # Matrix Assembly
  # UP = np.zeros((bc.n_pre_disp, 2))
  # for i in range(bc.n_pre_disp) :
  #   node = bc.disp_node[i]
  #   dof = bc.disp_dof[i]
  #   u = bc.disp_val[i]
  #   row = int(-(bc.eq_num[dof, node]) - 1)
  #   UP[row, dof] = u
  UP = np.zeros((bc.n_pre_disp, 1))
  for i in range(bc.n_pre_disp) :
    node = bc.disp_node[i]
    dof = bc.disp_dof[i]
    u = bc.disp_val[i]
    row = int(-(bc.eq_num[dof, node]) - 1)
    UP[row] = u

  PF = np.zeros((bc.n_dof, 1))
  for i in range(len(bc.force_val)) :
    node = bc.force_node[i]
    dof = bc.force_dof[i]
    f = bc.force_val[i]
    row = int(bc.eq_num[dof, node] - 1)
    if row >= 0 :
      PF[row, 0] += f

  PP = np.zeros((bc.n_pre_disp, 1))
  KPP = np.zeros((bc.n_pre_disp, bc.n_pre_disp))
  KPF = np.zeros((bc.n_pre_disp, bc.n_dof))
  KFP = np.zeros((bc.n_dof, bc.n_pre_disp))
  KFF = np.zeros((bc.n_dof, bc.n_dof))
  UUR = np.zeros((mesh.dim, mesh.n_node))
  PUR = np.zeros((mesh.dim, mesh.n_node))

  for elem_iter in range(mesh.n_elem) :
    
    K_el, P_el = element(elem_type, elem_iter, mesh, material, bc)

    for i in range(mesh.nodes_per_elem) : 
      i_node = mesh.elem_nodes[elem_iter][i]

      for i_dof in range(2) : # loop over 2 dof
        row = int(bc.eq_num[i_dof, i_node])
        ii = 2 * i + i_dof
        
        for j in range(mesh.nodes_per_elem) : #build columns
          j_node = mesh.elem_nodes[elem_iter][j]
          for j_dof in range(2) :
            col = int(bc.eq_num[j_dof, j_node])
            jj = 2 * j + j_dof

            if row > 0 :
              if col >  0 :
                KFF[row -1, col-1] += K_el[ii, jj]
              else :
                KFP[row-1, -col-1] += K_el[ii, jj]
            else :
              if col >  0 :
                KPF[-row-1, col-1] += K_el[ii, jj]
              else :
                KPP[-row-1, -col-1] += K_el[ii, jj]
        
        if row >= 0 :
          PF[row-1] += P_el[ii]
        else :
          PP[-row-1] += P_el[ii]

  # Build swapped KUR to align with UUR
  if efficient_opt :
    KUR = np.concatenate([np.concatenate([KPP, KPF], axis = 1),
                        np.column_stack([KFP, KFF])],
                        axis = 0)
    old_uur_idx = 0
    p_idx = 0
    f_idx = len(bc.disp_node)
    swap = [] # tuples of (UUR_node, K_row)
    for i_node in range(mesh.n_node) :
      for i_dof in range(2) :
        row = int(bc.eq_num[i_dof, i_node])
        if row > 0 :
          swap.append((old_uur_idx, f_idx))
          f_idx+=1
        else :
          swap.append((old_uur_idx, p_idx))
          p_idx+=1
        old_uur_idx +=1

    new_KUR =  KUR.copy()
    for pair in swap :
      temp_KUR = new_KUR
      # swap all first columns
      new_KUR[..., pair[0]] = temp_KUR[..., pair[1]]
      new_KUR[..., pair[1]] = temp_KUR[..., pair[0]]
      temp_KUR = new_KUR
      # swap whole rows
      new_KUR[pair[0], ...] = temp_KUR[pair[1], ...] 
      new_KUR[pair[1], ...] = temp_KUR[pair[0], ...] 
    new_KUR = scipy.sparse.csc_matrix(new_KUR)

    return FEA(UP, PF, PP, KPP, KPF, KFP, KFF, new_KUR)
  else :
    return FEA(UP, PF, PP, KPP, KPF, KFP, KFF, 0)




def solve_FEA(mesh, bc, fea) :  
  UUR = np.zeros((mesh.dim, mesh.n_node))
  PUR = np.zeros((mesh.dim, mesh.n_node))

  #Solve
  # UF = np.linalg.solve(fea.KFF, (fea.PF - np.matmul(fea.KFP, fea.UP)))
  UF = sparse_solve(scipy.sparse.csc_matrix(fea.KFF), scipy.sparse.csc_matrix((fea.PF - np.matmul(fea.KFP, fea.UP))))
  try :
    UF = UF.todense()
  except AttributeError :
    pass
  UF = UF.reshape((bc.n_dof,1))

  R = np.matmul(fea.KPP, fea.UP) + np.matmul(fea.KPF, UF) - fea.PP

  for i_node in range(mesh.n_node) :
    for i_dof in range(mesh.dim) :
      row = int(bc.eq_num[i_dof, i_node])
      if row > 0 :
        UUR[i_dof, i_node] = UF[row -1]
        PUR[i_dof, i_node] = fea.PF[row -1, 0]
      else :
        UUR[i_dof, i_node] = fea.UP[-row -1, 0]
        PUR[i_dof, i_node] = R[-row -1, 0]

  return Results(UF, R, UUR, PUR)

IndentationError: ignored

### Post Process, stress and strain

In [ ]:
def get_stress_gauss(elem_type, mesh, material, results) :
  """
  First Order Stress (r = 0) via Gauss Quad
  """
  if elem_type == "tri" :
    nodes_per_elem = 3
  else :
    nodes_per_elem = 4

  stress = np.zeros((nodes_per_elem, mesh.n_node))
  elem_stress = np.zeros((nodes_per_elem, mesh.n_elem))
  count  = np.zeros((1, mesh.n_node))

  for elem_iter in range(mesh.n_elem) :
    x_el = np.zeros((nodes_per_elem, mesh.dim))
    u_el = np.zeros((nodes_per_elem, mesh.dim))
    for i in range(nodes_per_elem) :
      node = mesh.elem_nodes[elem_iter][i]
      x_el[i, ...] = np.asarray(mesh.coords[node])
      u_el[i, ...] = results.UUR[...,node]

    # 1 Order GaussQuad
    if elem_type == "tri" :
      r = [1/3, 1/3]
      w = 2 + np.zeros((1,nodes_per_elem))
    else :
      r = [0, 0]
      w = 2 + np.zeros((1,nodes_per_elem))
    N, DN = shape_funct(elem_type, r)
    J = np.matmul(np.transpose(x_el), DN)
    detJ = J[0,0]*J[1,1] - J[0,1]*J[1,0]
    invJ = np.matrix([[J[1,1], -J[0,1]], [-J[1,0], J[0,0]]])/detJ
    
    B = np.matmul(DN, invJ)
    NhatToI = np.kron(np.transpose(N), np.eye(2))
    if elem_type == "tri" :
      BhatToI = np.zeros((3,6))
      BhatToI[0,0] = B[0][0,0]
      BhatToI[1,1] = B[0][0,1]
      BhatToI[0,2] = B[1][0,0]
      BhatToI[1,3] = B[1][0,1]
      BhatToI[0,4] = B[2][0,0]
      BhatToI[1,5] = B[2][0,1]
      BhatToI[2,0] = BhatToI[1,1]
      BhatToI[2,1] = BhatToI[0,0]
      BhatToI[2,2] = BhatToI[1,3]
      BhatToI[2,3] = BhatToI[0,2]
      BhatToI[2,4] = BhatToI[1,5]
      BhatToI[2,5] = BhatToI[0,4]
    else :
      BhatToI = np.kron(np.transpose(B), np.eye(2))

    Du = np.matmul(BhatToI, ((u_el).reshape(2*nodes_per_elem,1)))
    S = np.matmul(material.Ce[elem_iter] , Du)
    
    for j in range(nodes_per_elem) :
      elem_stress[..., elem_iter][j] += S[j]

    for i in range(nodes_per_elem) :
      col = mesh.elem_nodes[elem_iter][i]
      for j in range(nodes_per_elem) :
        stress[..., col][j] += S[j]
      count[...,col] += 1

  for i in range(mesh.n_node) :
    divisor = count[..., i][0]
    stress[..., i] = stress[..., i]/divisor
  
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    if elem_type == "quad" :
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    else :
      stress = np.stack([stress[0,...], stress[2,...], stress[2,...], stress[1,...]], axis = 0)
      elem_stress = np.stack([elem_stress[0,...], elem_stress[2,...], elem_stress[2,...], elem_stress[1,...]], axis = 0)
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    

  return np.column_stack([stress.T, von_mises.reshape(stress.T.shape[0], 1)]), np.column_stack([elem_stress.T, von_mises_elem.reshape(elem_stress.T.shape[0], 1)])

def get_dU_gauss(elem_type, mesh, uur) :
  """
  First Order Stress (r = 0) via Gauss Quad
  uur 2 x N_NODe
  """
  if elem_type == "tri" :
    nodes_per_elem = 3
  else :
    nodes_per_elem = 4

  stress = np.zeros((nodes_per_elem, mesh.n_node))
  elem_stress = np.zeros((nodes_per_elem, mesh.n_elem))
  count  = np.zeros((1, mesh.n_node))

  for elem_iter in range(mesh.n_elem) :
    x_el = np.zeros((nodes_per_elem, mesh.dim))
    u_el = np.zeros((nodes_per_elem, mesh.dim))
    for i in range(nodes_per_elem) :
      node = mesh.elem_nodes[elem_iter][i]
      x_el[i, ...] = np.asarray(mesh.coords[node])
      u_el[i, ...] = uur[node, ...]

    # 1 Order GaussQuad
    if elem_type == "tri" :
      r = [1/3, 1/3]
      w = 2 + np.zeros((1,nodes_per_elem))
    else :
      r = [0, 0]
      w = 2 + np.zeros((1,nodes_per_elem))
    N, DN = shape_funct(elem_type, r)
    J = np.matmul(np.transpose(x_el), DN)
    detJ = J[0,0]*J[1,1] - J[0,1]*J[1,0]
    invJ = np.matrix([[J[1,1], -J[0,1]], [-J[1,0], J[0,0]]])/detJ
    
    B = np.matmul(DN, invJ)
    NhatToI = np.kron(np.transpose(N), np.eye(2))
    if elem_type == "tri" :
      BhatToI = np.zeros((3,6))
      BhatToI[0,0] = B[0][0,0]
      BhatToI[1,1] = B[0][0,1]
      BhatToI[0,2] = B[1][0,0]
      BhatToI[1,3] = B[1][0,1]
      BhatToI[0,4] = B[2][0,0]
      BhatToI[1,5] = B[2][0,1]
      BhatToI[2,0] = BhatToI[1,1]
      BhatToI[2,1] = BhatToI[0,0]
      BhatToI[2,2] = BhatToI[1,3]
      BhatToI[2,3] = BhatToI[0,2]
      BhatToI[2,4] = BhatToI[1,5]
      BhatToI[2,5] = BhatToI[0,4]
    else :
      BhatToI = np.kron(np.transpose(B), np.eye(2))

    Du = np.matmul(BhatToI, ((u_el).reshape(2*nodes_per_elem,1)))
    
    for j in range(nodes_per_elem) :
      elem_stress[..., elem_iter][j] += Du[j]

    for i in range(nodes_per_elem) :
      col = mesh.elem_nodes[elem_iter][i]
      for j in range(nodes_per_elem) :
        stress[..., col][j] += Du[j]
      count[...,col] += 1

  for i in range(mesh.n_node) :
    divisor = count[..., i][0]
    stress[..., i] = stress[..., i]/divisor

  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    if elem_type == "quad" :
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    else :
      stress = np.stack([stress[0,...], stress[2,...], stress[2,...], stress[1,...]], axis = 0)
      elem_stress = np.stack([elem_stress[0,...], elem_stress[2,...], elem_stress[2,...], elem_stress[1,...]], axis = 0)
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    
  return np.column_stack([stress.T, von_mises.reshape(stress.T.shape[0], 1)]), np.column_stack([elem_stress.T, von_mises_elem.reshape(elem_stress.T.shape[0], 1)])

def get_stress_from_Du(elem_type, mesh, material, dU_elem) :
  """
  First Order Stress (r = 0) via Gauss Quad
  """
  if elem_type == "tri" :
    nodes_per_elem = 3
  else :
    nodes_per_elem = 4

  stress = np.zeros((nodes_per_elem, mesh.n_node))
  elem_stress = np.zeros((nodes_per_elem, mesh.n_elem))
  count  = np.zeros((1, mesh.n_node))

  for elem_iter in range(mesh.n_elem) :
    Du = dU_elem[elem_iter, ...].reshape(nodes_per_elem,1)
    S = np.matmul(material.Ce[elem_iter] , Du)
  
    for j in range(nodes_per_elem) :
      elem_stress[..., elem_iter][j] += S[j]

    for i in range(nodes_per_elem) :
      col = mesh.elem_nodes[elem_iter][i]
      for j in range(nodes_per_elem) :
        stress[..., col][j] += S[j]
      count[...,col] += 1

  for i in range(mesh.n_node) :
    divisor = count[..., i][0]
    stress[..., i] = stress[..., i]/divisor

  
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    if elem_type == "quad" :
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    else :
      stress = np.stack([stress[0,...], stress[2,...], stress[2,...], stress[1,...]], axis = 0)
      elem_stress = np.stack([elem_stress[0,...], elem_stress[2,...], elem_stress[2,...], elem_stress[1,...]], axis = 0)
      von_mises = (stress[0,...]**2 + stress[3,...]**2 - stress[0,...]*stress[3,...] + 3* stress[1,...]*stress[2,...] )**0.5
      von_mises_elem = (elem_stress[0,...]**2 + elem_stress[3,...]**2 - elem_stress[0,...]*elem_stress[3,...] + 3* elem_stress[1,...]*elem_stress[2,...] )**0.5
    
  return np.column_stack([stress.T, von_mises.reshape(stress.T.shape[0], 1)]), np.column_stack([elem_stress.T, von_mises_elem.reshape(elem_stress.T.shape[0], 1)])

def get_disp_err(s, m) :
  """ 
  Inputs : simulated UUR, modeled UUR as [N_NODE X 2]
  Returns : Nodal displacement error, avg, max, min
  """
  err = []
  err_abs = []
  for i in range(len(m)) :
      disp_s = (s[i][0]**2 + s[i][1]**2)**0.5
      disp_m = (m[i][0]**2 + m[i][1]**2)**0.5
      if disp_s != 0 :
          err.append((((disp_m - disp_s)/disp_s)))
          err_abs.append((((disp_m - disp_s)/disp_s)**2)**0.5)
      else :
          err.append((((disp_m - disp_s))))
          err_abs.append((((disp_m - disp_s))**2)**0.5)

  return err, sum(err_abs)/len(err), max(err), min(err)

def get_vect_err(s, m) :
  """ 
  Inputs : simulated vector, modeled stress col
  Returns : vect error
  """
  err = []
  err_abs = []
  for i in range(len(m)) :
      disp_s = s[i][0]
      disp_m = m[i][0]
      if disp_s != 0 :
          err.append((((disp_m - disp_s)/disp_s)))
          err_abs.append((((disp_m - disp_s)/disp_s)**2)**0.5)
      else :
          err.append((((disp_m - disp_s))))
          err_abs.append((((disp_m - disp_s))**2)**0.5)

  return err, sum(err_abs)/len(err), max(err), min(err)

### Commpliance

In [ ]:
def compliance_viaP(bc, results) :
  """ returns U' K U == U'PUR for values at force_node """
  compliance = 0
  for index in bc.force_node :
    compliance += results.UUR[0, index] * results.PUR[0, index]
    compliance += results.UUR[1, index] * results.PUR[1, index]
  compliance = compliance/2

  return compliance

def compliance_viaKUR(fea, results) :
  """ returns U' K U == U'PUR  """
  UUR_stack = results.UUR[..., 0:1]
  for i in range(1, len( results.UUR[0, ...])) :
    UUR_stack = np.concatenate([UUR_stack, results.UUR[..., i:i+1] ] )
  
  return np.dot( UUR_stack.T, np.matmul(fea.KUR, UUR_stack) )[0][0]

def compliance_viaKUR_mod(KUR, UUR) :
  """ returns U' K U == U'PUR  """
  UUR_stack = UUR[..., 0:1]
  for i in range(1, len( UUR[0, ...])) :
    UUR_stack = np.concatenate([UUR_stack, UUR[..., i:i+1] ] )
  
  return np.dot( UUR_stack.T, np.matmul(KUR, UUR_stack) )[0][0]

### Convergence

In [ ]:
def project_known_displacement(m1_a, m2_a, m1_b, m2_b, uur_a) :
  """ projects from [a], known fine-mesh, to [b], unknown coarse-mesh 
  ONLY WORKS if m1_a, m2_a are integer multiples of m1_b, m2_b
  """
  precision_base = 8
  d1_a = (bound_x[1] - bound_x[0])/m1_a
  d2_a = (bound_y[1] - bound_y[0])/m2_a
  d1_b = (bound_x[1] - bound_x[0])/m1_b
  d2_b = (bound_y[1] - bound_y[0])/m2_b

  n_node_a = (m1_a + 1) * (m2_a + 1)
  n_node_b = (m1_b + 1) * (m2_b + 1)

  # count coords row-wise, bottom to top, then apply noise
  coords_a = []
  for j in range(m2_a + 1) :
    for i in range(m1_a +1) :
      x = round(bound_x[0] + d1_a*i, precision_base)
      y = round(bound_y[0] + d2_a*j, precision_base)
      coords_a.append([x, y])
  coords_b = []
  for j in range(m2_b + 1) :
    for i in range(m1_b +1) :
      x = round(bound_x[0] + d1_b*i, precision_base)
      y = round(bound_y[0] + d2_b*j, precision_base)
      coords_b.append([x, y])

  # project displacement from a to b, there must be one node within one element of the fine mesh
  uur_b = np.zeros((n_node_b, 2))

  count = 0
  for (i_b, (x_b, y_b)) in enumerate(coords_b) :
    nearest_neighbors = []
    for (i_a, (x_a, y_a)) in enumerate(coords_a) :
      if (x_b == x_a) and (y_b == y_a) :
        uur_b[i_b] = uur_a[i_a]
        break

  return uur_b

# **Build graph from mesh**

`get_input_graph_informed(elem_type, mesh, bc)` \
*Uses projected values from coarse mesh*\
\
Globals :  [body_force_x, body_force_y]\
Nodes : [x, y, is_pre_x, is_pre_y, is_forced_x, is_forced_y, avg_density, is_surface, is_void]\
++ 
[1hot, u_x, u_y, P_x, P_y, du11, du21, du12, du22, Sh]__projection \

Edges : [x, y, del_x, del_y, length, elem_density, is_void]\
++
[d_u_x, d_u_y, d_P_x, d_P_y, d_du11, d_du21, d_du12, d_du22, d_Sh]__projection
\
\
`get_output_graph_informed(elem_type, input_graph, mesh, FEA, S, Du)` \
\*Adds delta_projected values from coarse mesh*\
\
Globals :  [body_force_x, body_force_y]\
Nodes : [u_x, u_y, P_x, P_y, du11, du21, du12, du22, Sh]\
++
[d_u_x, d_u_y, d_P_x, d_P_y, d_du11, d_du21, d_du12, d_du22, d_Sh] __delta projection  \

Edges : [del_length_x, del_length_y, length, delu11, delu21, delu12, delu22, delSh]
++\
[d_u_x, d_u_y, d_P_x, d_P_y, d_du11, d_du21, d_du12, d_du22, d_Sh]__delta projection 

In [ ]:
def dual_mesh_pre_process(mesh_0, mesh_1) :
  """ captures the shared nodes and elements from a coarse mesh_0 and fine mesh mesh_1"""
  M1 = mesh_0.m1
  M2 = mesh_0.m2
  m1 = mesh_1.m1
  m2 = mesh_1.m2
  m1_fact = int(m1/M1)
  m2_fact = int(m2/M2)
  father_elems = []
  for i in range(0, (m2-1) * (m1 + 1) + 1, m2_fact*(m1 + 1)) :
    for j in range(i, (i + m1), m1_fact) :
      elem_node = []
      elem_node.append(j)
      elem_node.append(j + 1*m1_fact)
      elem_node.append(j + m1*m2_fact + m1_fact + m2_fact)
      elem_node.append(j + m1*m2_fact + m2_fact)
      father_elems.append(elem_node)

  graph_nodes = []
  for elem in father_elems :
    graph_node = []
    for i in range(elem[0], elem[2] + 1, m1+1) :
      for j in range(i, i + 1 + m1_fact) :
        graph_node.append(j)
    graph_nodes.append(graph_node)

  return father_elems, graph_nodes

def multiscale_approximation(mesh_0, mesh_1, graph_nodes, output_graph_0) :
  """ projects a coarse approximation mesh_0, output_graph_0
  onto a fine mesh_1 using graph_nodes"""
  x_coarse, y_coarse = np.asarray(mesh_0.coords).T
  x_fine, y_fine = np.asarray(mesh_1.coords).T

  project_values = np.zeros((mesh_1.n_node, 9))
  for father_elem, fine_nodes in enumerate(graph_nodes) :
    coarse_nodes = mesh_0.elem_nodes[father_elem]
    for fine_node in fine_nodes :
      distance = []
      father_values = []
      for coarse_node in coarse_nodes :
        distance.append( ((x_fine[fine_node] - x_coarse[coarse_node])**2 + (y_fine[fine_node] - y_coarse[coarse_node])**2)**0.5 )
        father_values.append(output_graph_0["nodes"][coarse_node, 0::])
      scale_fine_to_coarse = (distance / max(distance))
      scale_fine_to_coarse = 1 - scale_fine_to_coarse / np.linalg.norm(scale_fine_to_coarse, ord = 2)
      scale_fine_to_coarse = (scale_fine_to_coarse / sum(scale_fine_to_coarse))
      
      project_values[fine_node] = np.matmul(np.asarray(father_values).T, scale_fine_to_coarse)

  return project_values

In [ ]:
def get_input_graph(elem_type, mesh, bc, element_density, surf_nodes, void_nodes) :
  """ 
  9 nodes: [x, y, 1ux, 1uy, 1Fx, 1Fy, rho, 1surf, 1void]
  7 edges: [x, y, del_x, del_y, length, rho_edges, 1void]
  """
  precision_base = 32
  #Globals
  # [body_force_x, body_force_y]
  globals = bc.elem_load[..., 0]

  #Nodes
  nodes = np.zeros((mesh.n_node, 9))

  node_density = np.zeros((mesh.n_node, 1))
  node_count = np.zeros((mesh.n_node, 1))
  for i, elem in enumerate(mesh.elem_nodes) :
    for node in elem :
      node_density[node] += element_density[i]
      node_count[node] += 1
  node_density /= node_count

  for i in range(mesh.n_node) :
    nodes[i, 0:2] = mesh.coords[i]
    nodes[i, 6] = node_density[i]
    nodes[i, 7] = surf_nodes[i]
    nodes[i, 8] = void_nodes[i]

  for i in range(len(bc.disp_node)) :
    if bc.disp_dof[i] == 0 :
      nodes[bc.disp_node[i], 2] = 1
    if bc.disp_dof[i] == 1 :
      nodes[bc.disp_node[i], 3] = 1

  for i in range(len(bc.force_node)) :
    if bc.force_dof[i] == 0 :
      nodes[bc.force_node[i], 4] = 1
    if bc.force_dof[i] == 1 :
      nodes[bc.force_node[i], 5] = 1

  # Edges 
  # [x, y, del_x, del_y, displacement]
  # Connected both ways along mesh boundaries
  senders = []
  receivers = []
  e_x = []
  e_y = []
  del_x = []
  del_y = []
  displacement = []

  send_rec = set()
  for elem in mesh.elem_nodes : 
    if elem_type == "tri" :
      if elem[0] < elem[1] :
        send_rec.add((elem[0], elem[1]))
      else :
        send_rec.add((elem[1], elem[0]))
      if elem[1] < elem[2] :
        send_rec.add((elem[1], elem[2]))
      else :
        send_rec.add((elem[2], elem[1]))
      if elem[0] < elem[2] :
        send_rec.add((elem[0], elem[2]))
      else :
        send_rec.add((elem[2], elem[0]))
    else :
      if elem[0] < elem[1] :
        send_rec.add((elem[0], elem[1]))
      else :
        send_rec.add((elem[1], elem[0]))
      if elem[1] < elem[2] :
        send_rec.add((elem[1], elem[2]))
      else :
        send_rec.add((elem[2], elem[1]))
      if elem[2] < elem[3] :
        send_rec.add((elem[2], elem[3]))
      else :
        send_rec.add((elem[3], elem[2]))
      if elem[0] < elem[3] :
        send_rec.add((elem[0], elem[3]))
      else :
        send_rec.add((elem[3], elem[0]))

  send_rec = list(send_rec)
  for i in range(len(send_rec)) :
    a = send_rec[i][0]
    b = send_rec[i][1]
    senders.append(a)
    senders.append(b)
    receivers.append(b)
    receivers.append(a)
    x = round( (mesh.coords[b][0] + mesh.coords[a][0])/2,
              precision_base)
    y = round( (mesh.coords[b][1] + mesh.coords[a][1])/2,
          precision_base)
    dx = round( (mesh.coords[b][0] - mesh.coords[a][0]),
              precision_base)
    dy = round( (mesh.coords[b][1] - mesh.coords[a][1]),
              precision_base)  
    displacement.append((dx**2 + dy**2) **0.5) 
    displacement.append(displacement[-1])    
    e_x.append(x)
    e_y.append(y)
    e_x.append(x)
    e_y.append(y)
    del_x.append(dx)
    del_x.append(-dx )
    del_y.append(dy)
    del_y.append(-dy)

  edges = np.zeros((len(senders), 7)) 
  for i in range(len(senders)) :
    edges[i,0] = e_x[i]
    edges[i,1] = e_y[i]
    edges[i,2] = del_x[i]
    edges[i,3] = del_y[i]
    edges[i,4] = displacement[i]
  
  for i, (send, rec) in enumerate(zip(senders, receivers)) :
    for elem_num, elem in enumerate(mesh.elem_nodes) :
      if (send in elem) and (rec in elem) : 
        edges[i,5] += element_density[elem_num] / 2
    if nodes[send, 8] == 1 or nodes[rec, 8] == 1 :
      edges[i,6] = 1

  input_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": receivers,
                        "senders": senders
                        }
  return input_graph



def get_input_graph_dual_informed(elem_type, mesh, bc, element_density, surf_nodes, void_nodes,
                             father_stat_list, father_projection_list, graph_node_list_projection,
                             mesh_projection, output_graph_projection) :
  
  """ 
  Uses infromation from the projection in attributes.
  19 nodes: [x, y, 1ux, 1uy, +-1Fx, +-1Fy, rho, 1surf, 1void, 
            1father_node, ux, uy, Fx, Fy, du_xx, du_yx, du_xy, du_yy, sig_vonmises]
  16 edges: [x, y, del_x, del_y, length, rho_edges, 1void, 
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  """
  precision_base = 32
  #Globals
  # [body_force_x, body_force_y]
  globals = bc.elem_load[..., 0]

  #Nodes
  nodes = np.zeros((mesh.n_node, 19))

  node_density = np.zeros((mesh.n_node, 1))
  node_count = np.zeros((mesh.n_node, 1))
  for i, elem in enumerate(mesh.elem_nodes) :
    for node in elem :
      node_density[node] += element_density[i]
      node_count[node] += 1
  node_density /= node_count

  for i in range(mesh.n_node) :
    nodes[i, 0:2] = mesh.coords[i]
    nodes[i, 6] = node_density[i]
    nodes[i, 7] = surf_nodes[i]
    nodes[i, 8] = void_nodes[i]

  for i in range(len(bc.disp_node)) :
    if bc.disp_dof[i] == 0 :
      nodes[bc.disp_node[i], 2] = 1
    if bc.disp_dof[i] == 1 :
      nodes[bc.disp_node[i], 3] = 1

  for i in range(len(bc.force_node)) :
    if bc.force_dof[i] == 0 :
      nodes[bc.force_node[i], 4] = bc.force_val[0]/(1e-6 + np.abs(bc.force_val[0]))
    if bc.force_dof[i] == 1 :
      nodes[bc.force_node[i], 5] = bc.force_val[1]/(1e-6 + np.abs(bc.force_val[1]))

  # Edges 
  # [x, y, del_x, del_y, displacement]
  # Connected both ways along mesh boundaries
  senders = []
  receivers = []
  e_x = []
  e_y = []
  del_x = []
  del_y = []
  displacement = []

  send_rec = set()
  for elem in mesh.elem_nodes : 
    if elem_type == "tri" :
      if elem[0] < elem[1] :
        send_rec.add((elem[0], elem[1]))
      else :
        send_rec.add((elem[1], elem[0]))
      if elem[1] < elem[2] :
        send_rec.add((elem[1], elem[2]))
      else :
        send_rec.add((elem[2], elem[1]))
      if elem[0] < elem[2] :
        send_rec.add((elem[0], elem[2]))
      else :
        send_rec.add((elem[2], elem[0]))
    else :
      if elem[0] < elem[1] :
        send_rec.add((elem[0], elem[1]))
      else :
        send_rec.add((elem[1], elem[0]))
      if elem[1] < elem[2] :
        send_rec.add((elem[1], elem[2]))
      else :
        send_rec.add((elem[2], elem[1]))
      if elem[2] < elem[3] :
        send_rec.add((elem[2], elem[3]))
      else :
        send_rec.add((elem[3], elem[2]))
      if elem[0] < elem[3] :
        send_rec.add((elem[0], elem[3]))
      else :
        send_rec.add((elem[3], elem[0]))

  send_rec = list(send_rec)
  for i in range(len(send_rec)) :
    a = send_rec[i][0]
    b = send_rec[i][1]
    senders.append(a)
    senders.append(b)
    receivers.append(b)
    receivers.append(a)
    x = round( (mesh.coords[b][0] + mesh.coords[a][0])/2,
              precision_base)
    y = round( (mesh.coords[b][1] + mesh.coords[a][1])/2,
          precision_base)
    dx = round( (mesh.coords[b][0] - mesh.coords[a][0]),
              precision_base)
    dy = round( (mesh.coords[b][1] - mesh.coords[a][1]),
              precision_base)  
    displacement.append((dx**2 + dy**2) **0.5) 
    displacement.append(displacement[-1])    
    e_x.append(x)
    e_y.append(y)
    e_x.append(x)
    e_y.append(y)
    del_x.append(dx)
    del_x.append(-dx )
    del_y.append(dy)
    del_y.append(-dy)

  edges = np.zeros((len(senders), 16)) 
  for i in range(len(senders)) :
    edges[i,0] = e_x[i]
    edges[i,1] = e_y[i]
    edges[i,2] = del_x[i]
    edges[i,3] = del_y[i]
    edges[i,4] = displacement[i]
  
  for i, (send, rec) in enumerate(zip(senders, receivers)) :
    for elem_num, elem in enumerate(mesh.elem_nodes) :
      if (send in elem) and (rec in elem) : 
        edges[i,5] += element_density[elem_num] / 2
    if nodes[send, 8] == 1 or nodes[rec, 8] == 1 :
      edges[i,6] = 1


  # MULTISCALE 
  for refined_node in flat_list(father_stat_list) :
    nodes[refined_node, 9] = 1.0
  nodes[..., 10:19] = multiscale_approximation(mesh_projection, mesh, graph_node_list_projection, output_graph_projection)

  for i, (send, rec) in enumerate(zip(senders, receivers)) :
    edges[i, 7:16] = nodes[rec, 10:19] - nodes[send, 10:19]


  input_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": receivers,
                        "senders": senders
                        }
  return input_graph

def get_input_graph_multiscale_informed(elem, input_graph_0, input_graph_1, output_graph_0, output_graph_1, graph_node_list, father_elem_node_list, mesh_0) :
  
  """ 
  Builds subgraphs using infromation from the projection in attributes.
  19 nodes: [x, y, 1ux, 1uy, +-1Fx, +-1Fy, rho, 1surf, 1void, 
            1father_node, ux, uy, Fx, Fy, du_xx, du_yx, du_xy, du_yy, sig_vonmises]
  16 edges: [x, y, del_x, del_y, length, rho_edges, 1void, 
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  """

  #Globals
  # [body_force_x, body_force_y]
  globals = input_graph_0["globals"]

  #Nodes
  n_nodes = len(graph_node_list)
  nodes = np.zeros((n_nodes, 19))
  for i, node in enumerate(graph_node_list) :
    nodes[i, ...] = input_graph_1["nodes"][node, ...]

  # for i, refined_node in enumerate(father_elem_node_list) :
  #   idx = graph_node_list.index(refined_node)
  #   nodes[idx, 9] = 1.0
    # nodes[idx, 10:19] = output_graph_0["nodes"][node, 0:9][mesh_0.elem_nodes[elem][i]]

  # Edges
  global_senders = []
  global_receivers = []
  global_edge_id_list = [] # global ids for 
  for edge, (a, b) in enumerate( zip(output_graph_1["senders"], output_graph_1["receivers"]) ) :
    if (a in graph_node_list) and (b in graph_node_list) :
      global_senders.append(a)
      global_receivers.append(b)
      global_edge_id_list.append(edge)

  new_senders = []
  new_receivers = []
  for s, r in zip(global_senders, global_receivers) :
    new_senders.append(graph_node_list.index(s))
    new_receivers.append(graph_node_list.index(r))

  n_edges = len(new_senders)
  edges = np.zeros((n_edges, 16))
  for local_id, global_edge in enumerate(global_edge_id_list) :
    edges[local_id, 0:16] = input_graph_1["edges"][global_edge]

  input_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": new_receivers,
                        "senders": new_senders
                        }

  return input_graph, global_edge_id_list





In [ ]:
def get_output_graph(elem_type, input_graph, mesh, results, Stress, Du) :

  """ 
  Uses high-fidelity analysis
  9 nodes: [ux, uy, Fx, Fy, du_xx, du_yx, du_xy, du_yy, sig_vonmises]
  8 edges: [del_ux, del_uy, final_length, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  """
  # Reuse Globals, senders, receivers
  # [body_force_x, body_force_y]
  globals = input_graph["globals"]
  senders = input_graph["senders"]
  receivers = input_graph["receivers"]

  #Nodes
  nodes = np.zeros((mesh.n_node, 9))

  for i in range(mesh.n_node) :
    nodes[i, 0] = results.UUR[0,i]
    nodes[i, 1] = results.UUR[1,i]
    nodes[i, 2] = results.PUR[0,i]
    nodes[i, 3] = results.PUR[1,i]
    nodes[i, 4] = Du[i, 0]
    nodes[i, 5] = Du[i, 1]
    nodes[i, 6] = Du[i, 2]
    nodes[i, 7] = Du[i, 3]
    nodes[i, 8] = Stress[i, 4]

    
  # Edges
  # [del_x, del_y, length, delS...]
  disp_coords = np.transpose(results.UUR) + mesh.coords
  del_x = []
  del_y = []
  displacement = []
  for i in range(len(senders)) :
    a = senders[i]
    b = receivers[i]

    dx = round( (disp_coords[b][0] - disp_coords[a][0]),
              precision_base)
    dy = round( (disp_coords[b][1] - disp_coords[a][1]),
              precision_base)  
    displacement.append((dx**2 + dy**2) **0.5) 
    del_x.append(dx)
    del_y.append(dy)

  edges = np.zeros((len(senders), 8)) 
  for i in range(len(senders)) :
    edges[i,0] = del_x[i] - input_graph["edges"][i,2] 
    edges[i,1] = del_y[i] - input_graph["edges"][i,3] 
    edges[i,2] = displacement[i]

  for i, (send, rec) in enumerate(zip(senders, receivers)) :
    edges[i, 3] = nodes[rec, 4] - nodes[send, 4]
    edges[i, 4] = nodes[rec, 5] - nodes[send, 5]
    edges[i, 5] = nodes[rec, 6] - nodes[send, 6]
    edges[i, 6] = nodes[rec, 7] - nodes[send, 7]
    edges[i, 7] = nodes[rec, 8] - nodes[send, 8]

  output_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": receivers,
                        "senders": senders
                        }
  return output_graph

def get_output_graph_informed(elem_type, input_graph, mesh, results, Stress, Du) :
  """ 
  Uses high-fidelity analysis, including the change in attribute relative to the low-fi analysis
  18 nodes: [ux, uy, Fx, Fy, du_xx, du_yx, du_xy, du_yy, sig_vonmises,
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  17 edges: [del_ux, del_uy, final_length, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises,
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  """
  # Reuse Globals, senders, receivers
  # [body_force_x, body_force_y]
  globals = input_graph["globals"]
  senders = input_graph["senders"]
  receivers = input_graph["receivers"]

  #Nodes
  nodes = np.zeros((mesh.n_node, 18))

  for i in range(mesh.n_node) :
    nodes[i, 0] = results.UUR[0,i]
    nodes[i, 1] = results.UUR[1,i]
    nodes[i, 2] = results.PUR[0,i]
    nodes[i, 3] = results.PUR[1,i]
    nodes[i, 4] = Du[i, 0]
    nodes[i, 5] = Du[i, 1]
    nodes[i, 6] = Du[i, 2]
    nodes[i, 7] = Du[i, 3]
    nodes[i, 8] = Stress[i, 4]
  nodes[..., 9:18] = nodes[..., 0:9] - input_graph["nodes"][..., 10:19]

    
  # Edges
  # [del_x, del_y, length, delS...]
  disp_coords = np.transpose(results.UUR) + mesh.coords
  del_x = []
  del_y = []
  displacement = []
  for i in range(len(senders)) :
    a = senders[i]
    b = receivers[i]

    dx = round( (disp_coords[b][0] - disp_coords[a][0]),
              precision_base)
    dy = round( (disp_coords[b][1] - disp_coords[a][1]),
              precision_base)  
    displacement.append((dx**2 + dy**2) **0.5) 
    del_x.append(dx)
    del_y.append(dy)

  edges = np.zeros((len(senders), 17)) 
  for i in range(len(senders)) :
    edges[i,0] = del_x[i] - input_graph["edges"][i,2] 
    edges[i,1] = del_y[i] - input_graph["edges"][i,3] 
    edges[i,2] = displacement[i]

  for i, (send, rec) in enumerate(zip(senders, receivers)) :
    edges[i, 3:8] = nodes[rec, 4:9] - nodes[send, 4:9]
    edges[i, 8:17] = nodes[rec, 9:18] - nodes[send, 9:18]

  output_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": receivers,
                        "senders": senders
                        }
  return output_graph

def get_output_graph_multiscale_informed(input_graph, output_graph_1, graph_node_list, edge_id_list) :
  """ 
  Builds sub-graphs using high-fidelity analysis, including the change in attribute relative to the low-fi analysis
  18 nodes: [ux, uy, Fx, Fy, du_xx, du_yx, du_xy, du_yy, sig_vonmises,
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  17 edges: [del_ux, del_uy, final_length, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises,
            del_ux, del_uy, del_Fx, del_Fy, del_du_xx, del_du_yx, del_du_xy, del_du_yy, del_sig_vonmises]
  """
  
  globals = input_graph["globals"]
  senders = input_graph["senders"]
  receivers = input_graph["receivers"]

  n_nodes = len(input_graph["nodes"][...,0])
  nodes = np.zeros((n_nodes, 18))
  for i, node in enumerate(graph_node_list) :
    nodes[i, ...] = output_graph_1["nodes"][node]

  edges = np.zeros((len(senders), 17)) 
  for i, edge in enumerate(edge_id_list) :
    edges[i, ...] = output_graph_1["edges"][edge] 

  output_graph = {"globals": np.asarray(globals).astype('float64') ,
                        "nodes": nodes.astype('float64') ,
                        "edges": edges.astype('float64') ,
                        "receivers": receivers,
                        "senders": senders
                        }
  return output_graph

# **GraphNetwork Model**
Sanchez-Gonzalez, A., Godwin, J., Pfaff, T., Ying, R., Leskovec, J., and Battaglia, P. W., 2020, "Learning to Simulate Complex Physics with Graph Networks," p. arXiv:2002.09405.

In [ ]:
# Sanchez-Gonzalez, A., Godwin, J., Pfaff, T., Ying, R., Leskovec, J., and Battaglia, P. W., 2020, 
# "Learning to Simulate Complex Physics with Graph Networks," p. arXiv:2002.09405.

import graph_nets as gn
import sonnet as snt
import tensorflow as tf


def build_mlp(
    hidden_size: int, num_hidden_layers: int, output_size: int) -> snt.Module:
  """Builds an MLP."""
  return snt.nets.MLP(
      output_sizes=[hidden_size] * num_hidden_layers + [output_size])


class EncodeProcessDecode(snt.AbstractModule):
  """Encode-Process-Decode function approximator for learnable simulator."""

  def __init__(
      self,
      latent_size: int,
      mlp_hidden_size: int,
      mlp_num_hidden_layers: int,
      num_message_passing_steps: int,
      output_size: int,
      name: str = "EncodeProcessDecode"):
    """Inits the model.
    Args:
      latent_size: Size of the node and edge latent representations.
      mlp_hidden_size: Hidden layer size for all MLPs.
      mlp_num_hidden_layers: Number of hidden layers in all MLPs.
      num_message_passing_steps: Number of message passing steps.
      output_size: Output size of the decode node representations as required
        by the downstream update function.
      name: Name of the model.
    """

    super().__init__(name=name)

    self._latent_size = latent_size
    self._mlp_hidden_size = mlp_hidden_size
    self._mlp_num_hidden_layers = mlp_num_hidden_layers
    self._num_message_passing_steps = num_message_passing_steps
    self._output_size = output_size

    with self._enter_variable_scope():
      self._networks_builder()

  def _build(self, input_graph: gn.graphs.GraphsTuple) -> tf.Tensor:
    """Forward pass of the learnable dynamics model."""

    # Encode the input_graph.
    latent_graph_0 = self._encode(input_graph)

    # Do `m` message passing steps in the latent graphs.
    latent_graph_m = self._process(latent_graph_0)

    # Decode from the last latent graph.
    return self._decode(latent_graph_m)

  def _networks_builder(self):
    """Builds the networks."""

    def build_mlp_with_layer_norm():
      mlp = build_mlp(
          hidden_size=self._mlp_hidden_size,
          num_hidden_layers=self._mlp_num_hidden_layers,
          output_size=self._latent_size)
      return snt.Sequential([mlp, snt.LayerNorm()])

    # The encoder graph network independently encodes edge and node features.
    encoder_kwargs = dict(
        edge_model_fn=build_mlp_with_layer_norm,
        node_model_fn=build_mlp_with_layer_norm)
    self._encoder_network = gn.modules.GraphIndependent(**encoder_kwargs)

    # Create `num_message_passing_steps` graph networks with unshared parameters
    # that update the node and edge latent features.
    # Note that we can use `modules.InteractionNetwork` because
    # it also outputs the messages as updated edge latent features.
    self._processor_networks = []
    for _ in range(self._num_message_passing_steps):
      self._processor_networks.append(
          gn.modules.InteractionNetwork(
              edge_model_fn=build_mlp_with_layer_norm,
              node_model_fn=build_mlp_with_layer_norm))

    # The decoder MLP decodes node latent features into the output size.
    self._decoder_network = build_mlp(
        hidden_size=self._mlp_hidden_size,
        num_hidden_layers=self._mlp_num_hidden_layers,
        output_size=self._output_size)

  def _encode(
      self, input_graph: gn.graphs.GraphsTuple) -> gn.graphs.GraphsTuple:
    """Encodes the input graph features into a latent graph."""

    # Copy the globals to all of the nodes, if applicable.
    if input_graph.globals is not None:
      broadcasted_globals = gn.blocks.broadcast_globals_to_nodes(input_graph)
      input_graph = input_graph.replace(
          nodes=tf.concat([input_graph.nodes, broadcasted_globals], axis=-1),
          globals=None)

    # Encode the node and edge features.
    latent_graph_0 = self._encoder_network(input_graph)
    return latent_graph_0

  def _process(
      self, latent_graph_0: gn.graphs.GraphsTuple) -> gn.graphs.GraphsTuple:
    """Processes the latent graph with several steps of message passing."""

    # Do `m` message passing steps in the latent graphs.
    # (In the shared parameters case, just reuse the same `processor_network`)
    latent_graph_prev_k = latent_graph_0
    for processor_network_k in self._processor_networks:
      latent_graph_k = self._process_step(
          processor_network_k, latent_graph_prev_k)
      latent_graph_prev_k = latent_graph_k

    latent_graph_m = latent_graph_k
    return latent_graph_m

  def _process_step(
      self, processor_network_k: snt.Module,
      latent_graph_prev_k: gn.graphs.GraphsTuple) -> gn.graphs.GraphsTuple:
    """Single step of message passing with node/edge residual connections."""

    # One step of message passing.
    latent_graph_k = processor_network_k(latent_graph_prev_k)

    # Add residuals.
    latent_graph_k = latent_graph_k.replace(
        nodes=latent_graph_k.nodes+latent_graph_prev_k.nodes,
        edges=latent_graph_k.edges+latent_graph_prev_k.edges)
    return latent_graph_k

  def _decode(self, latent_graph: gn.graphs.GraphsTuple) -> tf.Tensor:
    """Decodes from the latent graph."""
    return self._decoder_network(latent_graph.nodes), self._decoder_network(latent_graph.edges)

# **GraphNets Functions**

## Build Data from Graph Dicts

In [ ]:
def generate_raw_graph_tuple(input_graph_list, simulated_graph_list):
      """
      Processes graph_dicts into graphtuples
      """
      input_gt_list = []
      sim_gt_list = []
      for input_g in input_graph_list :
          input_gt = utils_tf.data_dicts_to_graphs_tuple([input_g])
          input_gt_list.append(input_gt)
      for sim_g in simulated_graph_list :
          sim_gt = utils_tf.data_dicts_to_graphs_tuple([sim_g])
          sim_gt_list.append(sim_gt)
          
      return input_gt_list, sim_gt_list


def generate_batched_graph_tuple(input_gt_list, sim_gt_list,
                                batch_size):
    """
    Concatenates the input graphtuples to create fixed batches
    """
    input_batch_list = []
    sim_batch_list = []
      
    for i in range(0, len(input_gt_list), batch_size - 1) :
        input_batch = input_gt_list[i]
        sim_batch = sim_gt_list[i]
        if batch_size > 1 :
            for j in range(i + 1, i + batch_size - 1) :
                input_0 = input_batch
                input_1 = input_gt_list[j]
                input_batch = (utils_tf.concat([input_0, input_1], axis = 0))

                sim_0 = sim_batch
                sim_1 = sim_gt_list[j]
                sim_batch = (utils_tf.concat([sim_0, sim_1], axis = 0))
        
        input_batch_list.append(input_batch)
        sim_batch_list.append(sim_batch)
    
    return input_batch_list, sim_batch_list

def generate_raw_data(input_graph_list, simulated_graph_list, batch_size) :
  """
  Generate raw graphtuple lists, no normalization
  """
  input_gt_list_tr, simulated_gt_list_tr  = generate_raw_graph_tuple(input_graph_list, simulated_graph_list)
  initial_conditions_tr, true_deformation_tr  = generate_batched_graph_tuple(input_gt_list_tr, simulated_gt_list_tr,
                                                                            batch_size_tr + 1)
  return initial_conditions_tr, true_deformation_tr

def build_input_graph_list_mixedBC(data_size, m1_min_max, m2_min_max, coord_noise_min_max, refine_min_max, vol_frac_min_max) :
    """
    Generates random mesh inputs
    """
    m1_list = []
    m2_list = []
    coord_noise_list = []
    refine_list = []
    vol_frac_list = []
    for i in range(data_size) :
      coord_noise_list.append(np.random.uniform(coord_noise_min_max[0], coord_noise_min_max[1]))
      m1_list.append(2*(np.random.randint(m1_min_max[0], m1_min_max[1])//2))
      m2_list.append(2*(np.random.randint(m2_min_max[0], m2_min_max[1])//2))
      refine_list.append((np.random.uniform(refine_min_max[0], refine_min_max[1])) )
      vol_frac_list.append((np.random.uniform(vol_frac_min_max[0], vol_frac_min_max[1])) )
        
    return m1_list, m2_list, coord_noise_list, refine_list, vol_frac_list

def generate_data(data_size, m1_min_max, m2_min_max, refine_min_max, vol_frac_min_max, density_min_max, 
                             noise_type, coord_noise_min_max, support_options, force_options, 
                       multi_fidelity_opt, KUR_opt, random_bound_opt, random_force_opt,
                         elem_type, base_bound_x, base_bound_y, E, v, applied_force_vals, force_dofs, body_force, multiscale_min_max_m1, multiscale_min_max_m2, force_scale,
                         scale_projection = True) :
  """
  Applies randomization to input, then simulates deformation, 
  returns lists of graph dicts via FEA
  """
  efficient_opt = KUR_opt 
  m1_list, m2_list, coord_noise_list, refine_list, vol_frac_list = build_input_graph_list_mixedBC(
      data_size, m1_min_max, m2_min_max, coord_noise_min_max, refine_min_max, vol_frac_min_max)
  raw_list = []
  input_graph_list = []
  simulated_graph_list = []
  recovery_structures = []
  for M1, M2, coord_noise, refine_fract, vol_frac in zip(m1_list, m2_list, coord_noise_list, refine_list, vol_frac_list) :
    
    # INIT
    if random_bound_opt :
      bound_x = [val*np.random.uniform(0.5, 2.) for val in list(base_bound_x)]
      bound_y_fact = np.random.uniform(0.5, 2.)
      bound_y = [val*bound_y_fact for val in list(base_bound_y)]
      bound_x = [round(val, 1) for val in bound_x]
      bound_y = [round(val, 1) for val in bound_y]

    else :
      bound_x, bound_y = base_bound_x, base_bound_y

    if random_force_opt :
      force_vals = [val*np.random.choice([-1,0,1]) for val in applied_force_vals]
      while (force_vals[0] == 0) and (force_vals[1] == 0) :
        force_vals = [val*np.random.choice([-1,0,1]) for val in applied_force_vals]
    else :
      force_vals = applied_force_vals

    support_option = np.random.choice(support_options)
    if support_option == "MBB" or support_option == "dual_cantilever" :
      force_option = np.random.choice(["top_distributed"])
      mesh_builder = build_mesh
      mesh_processor = dual_mesh_pre_process
    elif support_option == "L_bracket" :
      force_option = "side_center"
      mesh_builder = build_mesh_L_bracket
      mesh_processor = dual_mesh_pre_process_L_bracket
    else :
      force_option = np.random.choice(force_options)
      mesh_builder = build_mesh
      mesh_processor = dual_mesh_pre_process
      
    # SUB-DOMAIN INIT  
    mesh_0 = mesh_builder(elem_type, M1, M2, bound_x, bound_y,
                  "posi", 0.0, refine_fract)
    element_density_0 = [1]*mesh_0.n_elem
    surf_nodes_0  = [1]*mesh_0.n_node
    void_nodes_0  = [1]*mesh_0.n_node
    base_material_0 = assign_material(elem_type, mesh_0, E, v, element_density_0)
    bc_0 = assign_BC_option(mesh_0, force_vals, force_dofs, body_force,
                     support_option, force_option)
    fea_0 = assemble_matrices(elem_type, mesh_0, base_material_0, bc_0, efficient_opt)
    results_0 = solve_FEA(mesh_0, bc_0, fea_0)
    nodal_stress_0, elem_stress_0 = get_stress_gauss(elem_type, mesh_0, base_material_0, results_0)
    nodal_du_0, elem_du = get_dU_gauss(elem_type, mesh_0, results_0.UUR.T )
    input_graph_0 = get_input_graph(elem_type, mesh_0, bc_0, element_density_0, surf_nodes_0, void_nodes_0)
    output_graph_0 = get_output_graph(elem_type, input_graph_0, mesh_0, results_0, nodal_stress_0, nodal_du_0)

    # LOW-FIDELITY FEA
    if multi_fidelity_opt and scale_projection:
      m1_p = M1*np.random.randint(multiscale_min_max_m1[0], 1+multiscale_min_max_m1[1]//2)
      m2_p = M2*np.random.randint(multiscale_min_max_m2[0], 1+multiscale_min_max_m2[1]//2)
    elif multi_fidelity_opt and not scale_projection:
      m1_p = M1*2
      m2_p = M2*2
    else :
      m1_p, m2_p = M1, M2
    mesh_p = mesh_builder(elem_type, m1_p, m2_p, bound_x, bound_y,
                      "posi", 0.0, refine_fract)
    element_density_p = [1]*mesh_p.n_elem
    surf_nodes_p  = [1]*mesh_p.n_node
    void_nodes_p  = [1]*mesh_p.n_node
    base_material_p = assign_material(elem_type, mesh_p, E, v, element_density_p)
    bc_p = assign_BC_option(mesh_p, force_vals, force_dofs, body_force, support_option, force_option)
    fea_p = assemble_matrices(elem_type, mesh_p, base_material_p, bc_p, efficient_opt)
    results_p = solve_FEA(mesh_p, bc_p, fea_p)
    nodal_stress_p, elem_stress_p = get_stress_gauss(elem_type, mesh_p, base_material_p, results_p)
    nodal_du_p, elem_du = get_dU_gauss(elem_type, mesh_p, results_p.UUR.T )


    # HIGH-FIDELITY FEA
    if scale_projection :
      m1 = m1_p*np.random.randint(multiscale_min_max_m1[0], multiscale_min_max_m1[1])
      m2 = m2_p*np.random.randint(multiscale_min_max_m2[0], multiscale_min_max_m2[1])
    else :
      m1 = m1_p*np.random.randint(multiscale_min_max_m1[0], multiscale_min_max_m1[1])//2
      m2 = m2_p*np.random.randint(multiscale_min_max_m2[0], multiscale_min_max_m2[1])//2
    print("\nStat mesh: {} x {}".format(M1, M2), end = " || ")
    print("Father mesh: {} x {}".format(m1_p, m2_p), end = " || ")
    print("Child mesh: {} x {}".format(m1, m2), end = " || ")
    print("Force Vals: {:.4f}, {:.4f}".format(force_vals[0], force_vals[1]), end = " || ")
    print("{}, {}".format(support_option , force_option), end = " || ")
    print(bound_x, "x",  bound_y)
    mesh_1 = mesh_builder(elem_type, m1, m2, bound_x, bound_y,
                      "posi", 0.0, refine_fract)
    base_coords = copy.deepcopy(mesh_1.coords)
    if coord_noise > 0 or noise_type == "row" :
      mesh_1.coords = noise_mesh_MS(noise_type, coord_noise, mesh_0, mesh_1)
    else :
      print("WARNING: NO mesh noise")
    element_density_1 = [1]*mesh_1.n_elem
    base_density_1 = [1]*mesh_1.n_elem
    surf_nodes_1  = [1]*mesh_1.n_node
    void_nodes_1  = [1]*mesh_1.n_node
    base_material_1 = assign_material(elem_type, mesh_1, E, v, element_density_1)
    bc_1 = assign_BC_option(mesh_1, force_vals, force_dofs, body_force,
                     support_option, force_option)
    fea_1 = assemble_matrices(elem_type, mesh_1, base_material_1, bc_1, efficient_opt)
    results_1 = solve_FEA(mesh_1, bc_1, fea_1)
    nodal_stress_1, elem_stress_1 = get_stress_gauss(elem_type, mesh_1, base_material_1, results_1)
    nodal_du_1, _ = get_dU_gauss(elem_type, mesh_1, results_1.UUR.T )

    # BUILD GLOBAL GRAPHS
    father_elems_0, graph_nodes_0 = mesh_processor(mesh_0, mesh_1)
    father_elems, graph_nodes = mesh_processor(mesh_p, mesh_1)

    input_graph_0 = get_input_graph(elem_type, mesh_0, bc_0, element_density_0, surf_nodes_0, void_nodes_0)
    output_graph_0 = get_output_graph(elem_type, input_graph_0, mesh_0, results_0, nodal_stress_0, nodal_du_0)

    input_graph_p = get_input_graph(elem_type, mesh_p, bc_p, element_density_p, surf_nodes_p, void_nodes_p)
    output_graph_p = get_output_graph(elem_type, input_graph_p, mesh_p, results_p, nodal_stress_p, nodal_du_p)

    input_graph_1 = get_input_graph_dual_informed(elem_type, mesh_1, bc_1, element_density_1, surf_nodes_1, void_nodes_1,
                                father_elems_0, father_elems, graph_nodes,
                                mesh_p, output_graph_p) 
    output_graph_1 = get_output_graph_informed(elem_type, input_graph_1, mesh_1, results_1, nodal_stress_1, nodal_du_1)

    err_projection, avg_err_tst, max_tst, min_tst = get_disp_err(results_1.UUR.T, input_graph_1["nodes"][..., 10:12])
    print("Projection Error: avg {:.4f}, max {:.4f}, min {:.4f}".format(avg_err_tst, max_tst, min_tst))

 
    # CHECK FOR ERRORS
    z0 = np.asarray(results_1.UUR.T[..., 0])
    z1 = np.asarray(results_1.UUR.T[..., 1])
    double_check = np.maximum(np.amax(np.abs(z0)), np.amax(np.abs(z0)))
    if double_check > 0.1 or double_check == 0:
      print("FATAL ERROR, GARBAGE PRODUCED",double_check)
    else :
      print("Pass: ",double_check)
    
    # x, y = np.asarray(mesh_1.coords).T
    # plot_nodal_vect(x, y, z0, " ", 4e2)
    # plot_nodal_vect(x, y, z1, " ", 4e2)
    # plot_mesh(elem_type, False, " ", bound_x, bound_y,
    #           mesh_1, bc_1, results_1.UUR, elem_stress_1.T[-1,...])

    # RETURN
    if multi_fidelity_opt : 
      input_sub_graph_list = []
      output_sub_graph_list = []
      graph_edges = []
      for elem in range(mesh_0.n_elem) :
        graph_node_list = graph_nodes_0[elem]
        father_elem_node_list = father_elems_0[elem]

        ig, edge_id_list = get_input_graph_multiscale_informed(elem, input_graph_0, input_graph_1, output_graph_0, output_graph_1, 
                                                              graph_node_list, father_elem_node_list, mesh_0)
        og = get_output_graph_multiscale_informed(ig, output_graph_1, graph_node_list, edge_id_list)

        graph_edges.append(edge_id_list)
        input_sub_graph_list.append(ig)
        output_sub_graph_list.append(og)
      
      recovery_structures.append((output_sub_graph_list, graph_nodes_0, graph_edges, input_graph_0, output_graph_0, input_graph_1, output_graph_1))
      input_graph_list += input_sub_graph_list
      simulated_graph_list += output_sub_graph_list
      raw_list.append([mesh_1, bc_1, base_material_1, fea_1, results_1])
    
    else :
      recovery_structures.append((input_graph_0, output_graph_0, input_graph_1, output_graph_1, fea_1.KUR))
      raw_list.append([mesh_1, bc_1, base_material_1, fea_1, results_1])
      input_graph_list.append(input_graph_1)
      simulated_graph_list.append(output_graph_1)


  return raw_list, input_graph_list, simulated_graph_list, recovery_structures

def generate_convergence_data(M1_list, M2_list, m1p_list, m2p_list, m1_list, m2_list, refine_min_max, vol_frac_min_max, density_min_max, 
                             noise_type, coord_noise_min_max, support_options, force_options, 
                       multi_fidelity_opt, KUR_opt, random_bound_opt, random_force_opt,
                         elem_type, base_bound_x, base_bound_y, E, v, applied_force_vals, force_dofs, body_force, multiscale_min_max_m1, multiscale_min_max_m2, force_scale,
                         hole_radius = 0.0) :
  """
  Applies randomization to input, then simulates deformation, 
  returns lists of graph dicts via FEA
  """
  efficient_opt = KUR_opt 
  data_size = len(M1_list)
  m1_min_max, m2_min_max = (12,13), (6,7)
  _, _, coord_noise_list, refine_list, vol_frac_list = build_input_graph_list_mixedBC(
      data_size, m1_min_max, m2_min_max, coord_noise_min_max, refine_min_max, vol_frac_min_max)
  raw_list = []
  input_graph_list = []
  simulated_graph_list = []
  recovery_structures = []
  for i, (M1, M2, coord_noise, refine_fract, vol_frac) in enumerate(zip(M1_list, M2_list, coord_noise_list, refine_list, vol_frac_list)) :
    
    # INIT
    if random_bound_opt :
      bound_x = [val*np.random.uniform(0.5, 2.) for val in list(base_bound_x)]
      bound_y_fact = np.random.uniform(0.5, 2.)
      bound_y = [val*bound_y_fact for val in list(base_bound_y)]
      bound_x = [round(val, 1) for val in bound_x]
      bound_y = [round(val, 1) for val in bound_y]

    else :
      bound_x, bound_y = base_bound_x, base_bound_y

    if random_force_opt :
      force_vals = [val*np.random.choice([-1,0,1]) for val in applied_force_vals]
      while (force_vals[0] == 0) and (force_vals[1] == 0) :
        force_vals = [val*np.random.choice([-1,0,1]) for val in applied_force_vals]
    else :
      force_vals = applied_force_vals

    support_option = np.random.choice(support_options)
    if support_option == "MBB" or support_option == "dual_cantilever" :
      force_option = np.random.choice(["top_distributed"])
      mesh_builder = build_mesh
      mesh_processor = dual_mesh_pre_process
    elif support_option == "L_bracket" :
      force_option = "side_center"
      mesh_builder = build_mesh_L_bracket
      mesh_processor = dual_mesh_pre_process_L_bracket
    else :
      force_option = np.random.choice(force_options)
      mesh_builder = build_mesh
      mesh_processor = dual_mesh_pre_process

    m1_p, m2_p = m1p_list[i], m2p_list[i]
    m1, m2 =  m1_list[i], m2_list[i]
    print("\nStat mesh: {} x {}".format(M1, M2), end = " || ")
    print("Father mesh: {} x {}".format(m1_p, m2_p), end = " || ")
    print("Child mesh: {} x {}".format(m1, m2), end = " || ")
    print("Force Vals: {:.4f}, {:.4f}".format(force_vals[0], force_vals[1]), end = " || ")
    print("{}, {}".format(support_option , force_option), end = " || ")
    print(bound_x, "x",  bound_y)
    
    if hole_radius != 0 :
      mesh_builder = build_mesh_hole
      mesh_0, element_density_0 = mesh_builder(elem_type, M1, M2, bound_x, bound_y,
                  "posi", 0.0, refine_fract, radius=hole_radius)
      mesh_p, element_density_p = mesh_builder(elem_type, m1_p, m2_p, bound_x, bound_y,
                      "posi", 0.0, refine_fract, radius=hole_radius)
      mesh_1, element_density_1 = mesh_builder(elem_type, m1, m2, bound_x, bound_y,
                      "posi", 0.0, refine_fract, radius=hole_radius)
    else :
      mesh_0 = mesh_builder(elem_type, M1, M2, bound_x, bound_y,
                  "posi", 0.0, refine_fract)
      element_density_0 = [1]*mesh_0.n_elem
      mesh_p = mesh_builder(elem_type, m1_p, m2_p, bound_x, bound_y,
                      "posi", 0.0, refine_fract)
      element_density_p = [1]*mesh_p.n_elem
      mesh_1 = mesh_builder(elem_type, m1, m2, bound_x, bound_y,
                      "posi", 0.0, refine_fract)
      element_density_1 = [1]*mesh_1.n_elem

    # SUB-DOMAIN INIT  
    surf_nodes_0  = [1]*mesh_0.n_node
    void_nodes_0  = [1]*mesh_0.n_node
    base_material_0 = assign_material(elem_type, mesh_0, E, v, element_density_0)
    bc_0 = assign_BC_option(mesh_0, force_vals, force_dofs, body_force,
                     support_option, force_option)
    fea_0 = assemble_matrices(elem_type, mesh_0, base_material_0, bc_0, efficient_opt)
    results_0 = solve_FEA(mesh_0, bc_0, fea_0)
    nodal_stress_0, elem_stress_0 = get_stress_gauss(elem_type, mesh_0, base_material_0, results_0)
    nodal_du_0, elem_du = get_dU_gauss(elem_type, mesh_0, results_0.UUR.T )
    input_graph_0 = get_input_graph(elem_type, mesh_0, bc_0, element_density_0, surf_nodes_0, void_nodes_0)
    output_graph_0 = get_output_graph(elem_type, input_graph_0, mesh_0, results_0, nodal_stress_0, nodal_du_0)

    # LOW-FIDELITY FEA
    surf_nodes_p  = [1]*mesh_p.n_node
    void_nodes_p  = [1]*mesh_p.n_node
    base_material_p = assign_material(elem_type, mesh_p, E, v, element_density_p)
    bc_p = assign_BC_option(mesh_p, force_vals, force_dofs, body_force, support_option, force_option)
    fea_p = assemble_matrices(elem_type, mesh_p, base_material_p, bc_p, efficient_opt)
    results_p = solve_FEA(mesh_p, bc_p, fea_p)
    nodal_stress_p, elem_stress_p = get_stress_gauss(elem_type, mesh_p, base_material_p, results_p)
    nodal_du_p, elem_du = get_dU_gauss(elem_type, mesh_p, results_p.UUR.T )


    # HIGH-FIDELITY FEA
    base_coords = copy.deepcopy(mesh_1.coords)
    if coord_noise > 0 or noise_type == "row" :
      mesh_1.coords = noise_mesh_MS(noise_type, coord_noise, mesh_0, mesh_1)
    else :
      print("WARNING: NO mesh noise")
    base_density_1 = [1]*mesh_1.n_elem
    surf_nodes_1  = [1]*mesh_1.n_node
    void_nodes_1  = [1]*mesh_1.n_node
    base_material_1 = assign_material(elem_type, mesh_1, E, v, element_density_1)
    bc_1 = assign_BC_option(mesh_1, force_vals, force_dofs, body_force,
                     support_option, force_option)
    fea_1 = assemble_matrices(elem_type, mesh_1, base_material_1, bc_1, efficient_opt)
    results_1 = solve_FEA(mesh_1, bc_1, fea_1)
    nodal_stress_1, elem_stress_1 = get_stress_gauss(elem_type, mesh_1, base_material_1, results_1)
    nodal_du_1, _ = get_dU_gauss(elem_type, mesh_1, results_1.UUR.T )

    # BUILD GLOBAL GRAPHS
    father_elems_0, graph_nodes_0 = mesh_processor(mesh_0, mesh_1)
    father_elems, graph_nodes = mesh_processor(mesh_p, mesh_1)

    input_graph_0 = get_input_graph(elem_type, mesh_0, bc_0, element_density_0, surf_nodes_0, void_nodes_0)
    output_graph_0 = get_output_graph(elem_type, input_graph_0, mesh_0, results_0, nodal_stress_0, nodal_du_0)

    input_graph_p = get_input_graph(elem_type, mesh_p, bc_p, element_density_p, surf_nodes_p, void_nodes_p)
    output_graph_p = get_output_graph(elem_type, input_graph_p, mesh_p, results_p, nodal_stress_p, nodal_du_p)

    input_graph_1 = get_input_graph_dual_informed(elem_type, mesh_1, bc_1, element_density_1, surf_nodes_1, void_nodes_1,
                                father_elems_0, father_elems, graph_nodes,
                                mesh_p, output_graph_p) 
    output_graph_1 = get_output_graph_informed(elem_type, input_graph_1, mesh_1, results_1, nodal_stress_1, nodal_du_1)

    err_projection, avg_err_tst, max_tst, min_tst = get_disp_err(results_1.UUR.T, input_graph_1["nodes"][..., 10:12])
    print("Projection Error: avg {:.4f}, max {:.4f}, min {:.4f}".format(avg_err_tst, max_tst, min_tst))

 
    # CHECK FOR ERRORS
    z0 = np.asarray(results_1.UUR.T[..., 0])
    z1 = np.asarray(results_1.UUR.T[..., 1])
    double_check = np.maximum(np.amax(np.abs(z0)), np.amax(np.abs(z0)))
    if double_check > 0.1 or double_check == 0:
      print("FATAL ERROR, GARBAGE PRODUCED",double_check)
    else :
      print("Pass: ",double_check)
    
    # x, y = np.asarray(mesh_1.coords).T
    # plot_nodal_vect(x, y, z0, " ", 4e2)
    # plot_nodal_vect(x, y, z1, " ", 4e2)
    # plot_mesh(elem_type, False, " ", bound_x, bound_y,
    #           mesh_1, bc_1, results_1.UUR, elem_stress_1.T[-1,...])

    # RETURN
    if multi_fidelity_opt : 
      input_sub_graph_list = []
      output_sub_graph_list = []
      graph_edges = []
      for elem in range(mesh_0.n_elem) :
        graph_node_list = graph_nodes_0[elem]
        father_elem_node_list = father_elems_0[elem]

        ig, edge_id_list = get_input_graph_multiscale_informed(elem, input_graph_0, input_graph_1, output_graph_0, output_graph_1, 
                                                              graph_node_list, father_elem_node_list, mesh_0)
        og = get_output_graph_multiscale_informed(ig, output_graph_1, graph_node_list, edge_id_list)

        graph_edges.append(edge_id_list)
        input_sub_graph_list.append(ig)
        output_sub_graph_list.append(og)
      
      recovery_structures.append((output_sub_graph_list, graph_nodes_0, graph_edges, input_graph_0, output_graph_0, input_graph_1, output_graph_1))
      input_graph_list += input_sub_graph_list
      simulated_graph_list += output_sub_graph_list
      raw_list.append([mesh_1, bc_1, base_material_1, fea_1, results_1])
    
    else :
      recovery_structures.append((input_graph_0, output_graph_0, input_graph_1, output_graph_1, fea_1.KUR))
      raw_list.append([mesh_1, bc_1, base_material_1, fea_1, results_1])
      input_graph_list.append(input_graph_1)
      simulated_graph_list.append(output_graph_1)


  return raw_list, input_graph_list, simulated_graph_list, recovery_structures

## Standardization

In [ ]:
def fetch_stats(complete_dirname) :
  """
  load stats in from a saved list of graph_tuples
  """
  node_count = 0
  edge_count = 0

  item = []
  for load_idx in range(10) :
    with open('{}{}'.format(complete_dirname, load_idx), 'rb') as f :
      item += dill.load(f)

  # MEANS
  load_idx = 0
  for i, gt in enumerate(item) :
    if (load_idx == 0) and (i == 0) :
      node_sum = np.sum(gt.nodes, axis = 0)
      edge_sum = np.sum(gt.edges, axis = 0)
    else :
      node_sum += np.sum(gt.nodes, axis = 0)
      edge_sum += np.sum(gt.edges, axis = 0)
    node_count += gt.n_node
    edge_count += gt.n_edge
  node_means = node_sum / node_count
  edge_means = edge_sum / edge_count

  # STD DEVS
  for i, gt in enumerate(item) :
    if (load_idx == 0) and (i == 0) :
      node_sum = np.sum((gt.nodes - node_means)**2, axis = 0)
      edge_sum = np.sum((gt.edges - edge_means)**2, axis = 0)
    else :
      node_sum += np.sum((gt.nodes - node_means)**2, axis = 0)
      edge_sum += np.sum((gt.edges - edge_means)**2, axis = 0)
  node_stds = (node_sum / node_count)**0.5
  edge_stds = (edge_sum / edge_count)**0.5

  return node_means, node_stds, edge_means, edge_stds

def normalize_input_list_informed_multiscale(input_list, mean_list, std_list) :

  mean_array = np.asarray(mean_list)
  std_array = np.asarray(std_list)

  local_mean_list = []
  local_std_list = []
  new_edge_list = []
  local_edge_mean_list = []
  local_edge_std_list = []
  for input_gt in input_list :
    # USE GLOBAL STATS
    input_gt["nodes"][..., 0:2] = np.divide(np.subtract(input_gt["nodes"][..., 0:2],  
                                                      mean_array[0:2]), std_array[0:2]) 
    input_gt["edges"][..., 0:5] = np.divide(np.subtract(input_gt["edges"][..., 0:5],  
                                                      mean_array[2:7]), std_array[2:7])

    # COLLECT LOCAL STATS
    # father_means = np.max(input_gt["nodes"][..., 10:19], axis = 0) #np.mean
    # father_stds = np.min(input_gt["nodes"][..., 10:19], axis = 0) # np.std
    # father_edge_means = np.max(input_gt["edges"][..., 7:16], axis = 0)
    # father_edge_stds = np.min(input_gt["edges"][..., 7:16], axis = 0)
    father_means = np.mean(input_gt["nodes"][..., 10:19], axis = 0) 
    father_stds = np.std(input_gt["nodes"][..., 10:19], axis = 0) 
    father_edge_means = np.mean(input_gt["edges"][..., 7:16], axis = 0)
    father_edge_stds = np.std(input_gt["edges"][..., 7:16], axis = 0)
    local_mean_list.append(father_means)
    local_std_list.append(father_stds)
    local_edge_mean_list.append(father_edge_means)
    local_edge_std_list.append(father_edge_stds)
    
    # LOCAL NORMALIZATION
    for i in range(len(input_gt["nodes"][..., 0])) :
        # input_gt["nodes"][i, 10:12] = -0.5 +  np.divide(np.subtract(input_gt["nodes"][i, 10:12],  
        #                                                   father_stds[0:2]), (father_means[0:2] - father_stds[0:2]) )
        # input_gt["nodes"][i, 14:19] = -0.5 + np.divide(np.subtract(input_gt["nodes"][i, 14:19],  
        #                                                   father_stds[4:9]), father_means[4:9] - father_stds[4:9]) 
        input_gt["nodes"][i, 10:12] = np.divide(np.subtract(input_gt["nodes"][i, 10:12],  
                                                          father_means[0:2]), father_stds[0:2]) 
        input_gt["nodes"][i, 14:19] = np.divide(np.subtract(input_gt["nodes"][i, 14:19],  
                                                          father_means[4:9]), father_stds[4:9]) 

    for i in range(len(input_gt["edges"][..., 0])) :
        input_gt["edges"][i, 7:9] = -0.5 + np.divide(np.subtract(input_gt["edges"][i, 7:9],  
                                                          father_edge_stds[0:2]), father_edge_means[0:2] - father_edge_stds[0:2]) 
        input_gt["edges"][i, 11:16] = -0.5 + np.divide(np.subtract(input_gt["edges"][i, 11:16],  
                                                  father_edge_stds[4:9]), father_edge_means[4:9] -  father_edge_stds[4:9]) 
        # input_gt["edges"][i, 7:9] = np.divide(np.subtract(input_gt["edges"][i, 7:9],  
        #                                                   father_edge_means[0:2]), father_edge_stds[0:2]) 
        # input_gt["edges"][i, 11:16] = np.divide(np.subtract(input_gt["edges"][i, 11:16],  
        #                                           father_edge_means[4:9]), father_edge_stds[4:9]) 

  
  return input_list, local_mean_list, local_std_list, local_edge_mean_list, local_edge_std_list

def local_standard_target(target_graph_list, local_means, local_stds) :

  for idx, target in enumerate(target_graph_list) :
    # LOCAL NORMALIZATION
    for i in range(len(target["nodes"][..., 0])) :
      # target["nodes"][i, 0:2] = -0.5 + (target["nodes"][i, 0:2] - local_stds[idx][0:2])/ (local_means[idx][0:2] - local_stds[idx][0:2])
      target["nodes"][i, 0:2] = (target["nodes"][i, 0:2] - local_means[idx][0:2])/ (local_stds[idx][0:2])
  return target_graph_list

## Error and Processing from GN

In [ ]:
def get_disp_err(s, m) :
  """ 
  Inputs : simulated UUR, modeled UUR as [N_NODE X 2]
  Returns : Nodal displacement error, avg, max, min
  """
  err = []
  err_abs = []
  for i in range(len(m)) :
      disp_s = (s[i][0]**2 + s[i][1]**2)**0.5
      disp_m = (m[i][0]**2 + m[i][1]**2)**0.5
      if disp_s != 0 :
          err.append((((disp_m - disp_s)/disp_s)))
          err_abs.append((((disp_m - disp_s)/disp_s)**2)**0.5)
      else :
          err.append((((disp_m - disp_s))))
          err_abs.append((((disp_m - disp_s))**2)**0.5)

  return err, sum(err_abs)/len(err), max(err), min(err)

def get_vect_err(s, m) :
  """ 
  Inputs : simulated vector, modeled stress col
  Returns : vect error
  """
  err = []
  err_abs = []
  for i in range(len(m)) :
      disp_s = s[i][0]
      disp_m = m[i][0]
      if disp_s != 0 :
          err.append((((disp_m - disp_s)/disp_s)))
          err_abs.append((((disp_m - disp_s)/disp_s)**2)**0.5)
      else :
          err.append((((disp_m - disp_s))))
          err_abs.append((((disp_m - disp_s))**2)**0.5)

  return err, sum(err_abs)/len(err), max(err), min(err)


def make_all_runnable_in_session(*args):
  """Apply make_runnable_in_session to an iterable of graphs."""
  return [utils_tf.make_runnable_in_session(a) for a in args]



def get_node_trajectories(rollout_array, batch_size, idx0, idx1):  # pylint: disable=redefined-outer-name
  return np.split(rollout_array[..., idx0:idx1], batch_size, axis=0)

# **Plots**

In [ ]:
from matplotlib import cm
from matplotlib import colors

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 16}

mpl.rc('font', **font)

def get_colors(inp, colormap):
    vmin = np.min(inp)
    vmax = np.max(inp)
    norm = plt.Normalize(vmin, vmax)
    return norm, colormap(norm(inp))

def plot_mesh(elem_type, annotations, alternate_title, bound_x, bound_y,
              mesh, bc, uur, elem_stress, plot_bc = True) :
  # Plot Mesh
  fig = plt.figure(1, figsize=(50, 10))
  ax = fig.add_subplot(1, 2, 1)
  plt.xticks((np.arange( bound_x[0], bound_x[1] + 2*mesh.d1, step=5 *mesh.d1)) )
  plt.yticks((np.arange( bound_y[0], bound_y[1] + 2*mesh.d2, step=2 *mesh.d2)) )
  # ax.grid(True, which = 'both')
  ax.set_title("Mesh: " + alternate_title) 
  ax.set_ylabel('y')
  ax.set_xlabel('x')

  #nodes
  x, y = np.asarray(mesh.coords).T
  ax.scatter(x, y, c = 'red', zorder = 10)
  if annotations is True :
    for i in range(mesh.n_node) :
      ax.annotate(str(i), (x[int(i)], y[int(i)]), size = 8)

  #elements
  for elem in mesh.elem_nodes :
    if elem_type == "tri" :
      elem_coords= [mesh.coords[elem[0]], mesh.coords[elem[1]], 
                          mesh.coords[elem[2]], mesh.coords[elem[0]]]
    else :
      elem_coords= [mesh.coords[elem[0]], mesh.coords[elem[1]], 
                          mesh.coords[elem[2]], mesh.coords[elem[3]], mesh.coords[elem[0]]]
    x, y = np.asarray(elem_coords).T
    ax.plot(x, y, 'k')
  if annotations is True :
    for elem in mesh.elem_nodes : 
      x_bar = 0
      y_bar = 0
      for i in elem :
        x_bar += mesh.coords[i][0]
        y_bar += mesh.coords[i][1]
      if elem_type == "tri" :
        x, y = np.asarray([x_bar/3, y_bar/3]).T
      else :
        x, y = np.asarray([x_bar/4, y_bar/4]).T
      ax.annotate(mesh.elem_nodes.index(elem), (x, y), 
                  size = 8, ha='center', va='bottom')
      
  #boundary conditions
  for i in bc.eq_num[0].tolist() :
    if i < 0 :
      node = (bc.eq_num[0].tolist().index(i))
      x, y = np.asarray(mesh.coords[node]).T
      ax.plot(x, y, '>k', markersize = 12, fillstyle = 'none', mew =2)
  for i in bc.eq_num[1].tolist() :
    if i < 0 :
      node = (bc.eq_num[1].tolist().index(i))
      x, y = np.asarray(mesh.coords[node]).T
      ax.plot(x, y, '^k', markersize = 12, fillstyle = 'none', mew =2)
  for i in bc.force_node :
    x, y = np.asarray(mesh.coords[i]).T
    ax.plot(x, y, 'or', markersize = 16, fillstyle = 'none', mew =2)



  ################################################################################
  #Plot deformed condition 
  norm, color_array = get_colors(elem_stress, plt.cm.jet)
  new_coords = uur.T + np.array(mesh.coords)
  ax = fig.add_subplot(1, 2, 2)

  #Undeformed Nodes
  x, y = np.asarray(mesh.coords).T
  plt.xticks((np.arange( bound_x[0], bound_x[1] + 2*mesh.d1, step=5 *mesh.d1)) )
  plt.yticks((np.arange( bound_y[0], bound_y[1] + 2*mesh.d2, step=2 *mesh.d2)) )
  # ax.grid(True, which = 'both')
  ax.set_title("Deformed Mesh with Von Mises Stress") 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  ax.scatter(x, y, c = 'gray')

  #Undeformed Elements
  for elem in mesh.elem_nodes :
    if elem_type == "tri" :
      elem_coords= [mesh.coords[elem[0]], mesh.coords[elem[1]], 
                          mesh.coords[elem[2]], mesh.coords[elem[0]]]
    else :
      elem_coords= [mesh.coords[elem[0]], mesh.coords[elem[1]], 
                          mesh.coords[elem[2]], mesh.coords[elem[3]], mesh.coords[elem[0]]]
    x, y = np.asarray(elem_coords).T
    ax.plot(x, y, 'gray', linewidth = 1)


  #Deformed Nodes
  x, y = (new_coords).T
  ax.scatter(x, y, c = 'black', zorder = 10)

  #Deformed Elements
  for i, elem in enumerate(mesh.elem_nodes ): 
    if elem_type == "tri" :
      elem_coords= [new_coords[elem[0]], new_coords[elem[1]], 
                          new_coords[elem[2]], new_coords[elem[0]]]
    else :
      elem_coords= [new_coords[elem[0]], new_coords[elem[1]], 
                          new_coords[elem[2]], new_coords[elem[3]], new_coords[elem[0]]]
    x, y = np.asarray(elem_coords).T
    ax.plot(x, y, 'red', linewidth = 3)
    if plot_bc :
      plt.fill_between(x, y, color = color_array[i])

  cax, _ = mpl.colorbar.make_axes(ax) 
  cb2 = mpl.colorbar.ColorbarBase(cax, cmap=plt.cm.jet,norm=norm) 
  
  #Deformed Boundary Conditions
  if plot_bc :
    for i in bc.eq_num[0].tolist() :
      if i < 0 :
        node = (bc.eq_num[0].tolist().index(i))
        x, y = np.asarray(new_coords[node]).T
        ax.plot(x, y, '>k', markersize = 12, fillstyle = 'none', mew =2)
    for i in bc.eq_num[1].tolist() :
      if i < 0 :
        node = (bc.eq_num[1].tolist().index(i))
        x, y = np.asarray(new_coords[node]).T
        ax.plot(x, y, '^k', markersize = 12, fillstyle = 'none', mew =2)
    for i in bc.force_node :
      x, y = np.asarray(new_coords[i]).T
      ax.plot(x, y, 'or', markersize = 16, fillstyle = 'none', mew =2)
  plt.show()
  fig.clf()

def plot_nodal_vect(x, y, z, title, size) :
  fig = plt.figure(1, figsize=(10, 5))
  # Plot Mesh
  ax = fig.add_subplot(1, 1, 1)
  ax.set_title(title) 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  #nodes
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin= z.min(), vmax= z.max())
  plt.colorbar()
  plt.show()
  fig.clf()

def plot_elem_attr(elem_type, title, mesh, elem_array) :
  
  fig = plt.figure(1, figsize=(50, 20))
  size = 1e3 * 1000 / mesh.n_node
  
  elem_coords = []
  if elem_type == "tri" :
    nodes_per_elem = 3
    z_plot = [elem_array.T[0, ...], elem_array.T[2, ...], elem_array.T[2, ...], elem_array.T[1, ...]]
  else :
    nodes_per_elem = 4
    z_plot = [elem_array.T[0, ...], elem_array.T[1, ...], elem_array.T[2, ...], elem_array.T[3, ...]]
  for elem in mesh.elem_nodes :
    avg_x = 0
    avg_y = 0
    for node in elem:
      avg_x += mesh.coords[node][0] / nodes_per_elem
      avg_y += mesh.coords[node][1] / nodes_per_elem
    elem_coords.append([avg_x, avg_y])
  

  # 11
  z = z_plot[0]
  ax = fig.add_subplot(2, 2, 1)
  ax.set_title(title + " 11") 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  #nodes
  x, y = np.asarray(elem_coords).T
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin=z.min(), vmax=z.max())
  plt.colorbar()

  # 21
  z = z_plot[1]
  ax = fig.add_subplot(2, 2, 2)
  ax.set_title(title + " 21") 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  #nodes
  x, y = np.asarray(elem_coords).T
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin=z.min(), vmax=z.max())
  plt.colorbar()

  # 12
  z = z_plot[2]
  ax = fig.add_subplot(2, 2, 3)
  ax.set_title(title + " 12") 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  #nodes
  x, y = np.asarray(elem_coords).T
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin=z.min(), vmax=z.max())
  plt.colorbar()

  # 22
  z = z_plot[3]
  ax = fig.add_subplot(2, 2, 4)
  ax.set_title(title + " 22") 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  #nodes
  x, y = np.asarray(elem_coords).T
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin=z.min(), vmax=z.max())
  plt.colorbar()
  plt.show()
  fig.clf()

def plot_capped_nodal_vect(x, y, z, title, size, cap_min, cap_max) :
  fig = plt.figure(1, figsize=(50, 10))
  # Plot Mesh
  delta = 0.1
  ax = fig.add_subplot(1, 2, 1)
  ax.set_title(title) 
  ax.set_ylabel('y')
  ax.set_xlabel('x')
  ax.set_xlim(x.min() - delta, x.max() + delta)
  ax.set_ylim(y.min() - delta, y.max() + delta)
  #nodes
  plt.scatter(x, y, c=z, marker = 's', s = size,
              cmap='jet',vmin= cap_min, vmax= cap_max)
  plt.colorbar()
  plt.show()
  fig.clf()